## 1.Download the datasets from UCSC website

### 1.1 Download the multi-omics data

* Parse the data from the UCSC Xena website in PANCAN cohort:
https://xenabrowser.net/datapages/?cohort=TCGA%20Pan-Cancer%20(PANCAN)&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443

<!-- * Copy number (gene-level) - gene-level copy number (gistic2_thresholded)
    * Dataset: https://xenabrowser.net/datapages/?dataset=TCGA.PANCAN.sampleMap%2FGistic2_mutation_Gistic2_all_thresholded.by_genes&host=https%3A%2F%2Ftcga.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443 -->
* Copy number - somatic mutation (SNP and INDEL) - Gene level non-silent mutation
    * Dataset: https://xenabrowser.net/datapages/?dataset=mc3.v0.2.8.PUBLIC.nonsilentGene.xena&host=https%3A%2F%2Fpancanatlas.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443

* DNA methylation (Methylation450K)
    * Dataset: https://xenabrowser.net/datapages/?dataset=jhu-usc.edu_PANCAN_HumanMethylation450.betaValue_whitelisted.tsv.synapse_download_5096262.xena&host=https%3A%2F%2Fpancanatlas.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443
    * ID Map: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GPL16304

* Gene expression RNAseq - TOIL RSEM fpkm
    * Dataset: https://xenabrowser.net/datapages/?dataset=tcga_RSEM_gene_fpkm&host=https%3A%2F%2Ftoil.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443
    
* Protein expression - RPPA
    * Dataset: https://xenabrowser.net/datapages/?dataset=TCGA-RPPA-pancan-clean.xena&host=https%3A%2F%2Fpancanatlas.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443

### 1.2 Download the clinical data

### 1.2 Download the clinical data

* Phenotype - Curated clinical data
    * Dataset: https://xenabrowser.net/datapages/?dataset=Survival_SupplementalTable_S1_20171025_xena_sp&host=https%3A%2F%2Fpancanatlas.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443

* Phenotype - Immune subtype
    * Dataset: https://xenabrowser.net/datapages/?dataset=Subtype_Immune_Model_Based.txt&host=https%3A%2F%2Fpancanatlas.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443

* Phenotype - Molecular subtype
    * Dataset: https://xenabrowser.net/datapages/?dataset=TCGASubtype.20170308.tsv&host=https%3A%2F%2Fpancanatlas.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443

* Phenotype - sample type and primary disease
    * Dataset: https://xenabrowser.net/datapages/?dataset=TCGA_phenotype_denseDataOnlyDownload.tsv&host=https%3A%2F%2Fpancanatlas.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443


## 2.Read the files

### 2.1 Read DNA methylation data

In [1]:
import pandas as pd

In [2]:
methylation_value = pd.read_csv('./UCSC-raw/jhu-usc.edu_PANCAN_HumanMethylation450.betaValue_whitelisted.tsv.synapse_download_5096262.xena', delimiter='\t')

In [3]:
methylation_value

,sample,TCGA-OR-A5J1-01,TCGA-OR-A5J2-01,TCGA-OR-A5J3-01,TCGA-OR-A5J4-01,TCGA-OR-A5J5-01,TCGA-OR-A5J6-01,TCGA-OR-A5J7-01,TCGA-OR-A5J8-01,TCGA-OR-A5J9-01,...,TCGA-YR-A95A-01,TCGA-ZD-A8I3-01,TCGA-ZH-A8Y1-01,TCGA-ZH-A8Y2-01,TCGA-ZH-A8Y4-01,TCGA-ZH-A8Y5-01,TCGA-ZH-A8Y6-01,TCGA-ZH-A8Y8-01,TCGA-ZU-A8S4-01,TCGA-ZU-A8S4-11
0,cg00000029,0.119877,0.107120,0.060752,0.157005,0.534426,0.117144,0.803340,0.853057,0.611181,...,0.586259,0.585331,0.161617,0.106717,0.218970,0.064768,0.102256,0.168150,0.575601,0.407320
1,cg00000165,0.903200,0.818523,0.093014,0.856304,0.928812,0.115994,0.904277,0.174365,0.898031,...,0.536294,0.566928,0.418702,0.320658,0.792981,0.127150,0.083132,0.133048,0.299671,0.129993
2,cg00000236,0.879704,0.256479,0.253639,0.940341,0.931214,0.413085,0.926751,0.940353,0.919704,...,0.910524,0.701900,0.779136,0.859169,0.889698,0.932694,0.848870,0.798637,0.897517,0.823814
3,cg00000289,0.766316,0.674323,0.750682,0.842987,0.766848,0.584756,0.723257,0.791033,0.833011,...,0.643099,0.328733,0.418107,0.399530,0.560327,0.558825,0.529123,0.455987,0.683475,0.514753
4,cg00000292,0.409997,0.552000,0.851327,0.847317,0.761502,0.782867,0.918350,0.870836,0.887851,...,0.855259,0.873919,0.523226,0.849727,0.601534,0.604496,0.857456,0.853369,0.810656,0.713617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396060,rs9363764,0.949515,0.152343,0.060605,0.537235,0.068372,0.546527,0.716253,0.949553,0.918098,...,0.555305,0.834596,0.889483,0.177781,0.787535,0.949106,0.039839,0.055593,0.934815,0.921423
396061,rs939290,0.517026,0.511248,0.968356,0.119577,0.970206,0.676457,0.025504,0.969516,0.928915,...,0.968385,0.903384,0.039378,0.948194,0.856111,0.029937,0.950910,0.414340,0.560110,0.562216
396062,rs951295,0.965846,0.214991,0.199349,0.845229,0.131683,0.040208,0.213107,0.972727,0.529565,...,0.536218,0.505198,0.049942,0.546060,0.038895,0.537847,0.545609,0.965346,0.577749,0.558346
396063,rs966367,0.943978,0.926279,0.056009,0.447347,0.050560,0.938875,0.347014,0.169791,0.035946,...,0.956214,0.436520,0.421785,0.607085,0.894245,0.866823,0.834397,0.040033,0.541568,0.477821


### 2.2 Read Platform annotations for 450k methylation 

In [4]:
import pandas as pd
# Reading and processing basic data
try:
    annotation = pd.read_table('./UCSC-raw/GPL16304-47833.txt', delimiter='\t')
    annotation['Distance_closest_TSS'] = annotation['Distance_closest_TSS'].astype(int)
    annotation = annotation[~annotation['Closest_TSS'].apply(lambda x: len(str(x).split(';')) > 1)]
except ValueError as e:
    print(f"Unable to convert 'Closest_TSS' column to integer: {e}")
    problematic_rows = annotation['Distance_closest_TSS'].apply(lambda x: not str(x).isnumeric())
    print("Problematic rows:")
    print(annotation.loc[problematic_rows])

annotation

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_22064\1287874560.py:4: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  annotation = pd.read_table('./UCSC-raw/GPL16304-47833.txt', delimiter='\t')


,ID,MAPINFO-1,MAPINFO+1,Probe_start,Probe_end,Target CpG SNP,n_target CpG SNP,SNPprobe,n_SNPprobe,HIL_CpG_class,...,AlleleA_Hits,AlleleB_Hits,XY_Hits,Autosomal_Hits,Closest_TSS,Closest_TSS_1,Distance_closest_TSS,Closest_TSS_gene_name,Closest_TSS_Transcript,SPOT_ID
0,cg00000029,53468111,53468113,53468112,53468162,NaN,NaN,NaN,NaN,HC,...,1,0,XY_NO,A_NO,53468350,53468351,-238,RBL2,NM_005611,cg00000029
1,cg00000108,37459205,37459207,37459206,37459256,NaN,NaN,rs9857774,1.0,LC,...,1,0,XY_NO,A_NO,37458757,37458758,449,C3orf35,CCDS46792,cg00000108
2,cg00000109,171916036,171916038,171916037,171916087,NaN,NaN,rs9864492,1.0,LC,...,1,0,XY_NO,A_NO,171851260,171851261,64777,FNDC3B,AY358367,cg00000109
3,cg00000165,91194673,91194675,91194624,91194674,NaN,NaN,rs76771611,1.0,ICshore,...,1,0,XY_NO,A_NO,91182793,91182794,-11880,BARHL2,NM_020063,cg00000165
4,cg00000236,42263293,42263295,42263244,42263294,NaN,NaN,NaN,NaN,IC,...,1,0,XY_NO,A_NO,42251727,42251728,11567,VDAC3,CCDS47850,cg00000236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485507,ch.X.97129969R,97243312,97243314,97243313,97243363,NaN,NaN,NaN,NaN,LC,...,1,0,XY_NO,A_NO,97607750,97607751,-364437,Mir_340,.,ch.X.97129969R
485508,ch.X.97133160R,97246503,97246505,97246504,97246554,NaN,NaN,NaN,NaN,LC,...,2,0,XY_NO,A_YES,97607750,97607751,-361246,Mir_340,.,ch.X.97133160R
485509,ch.X.97651759F,97765102,97765104,97765103,97765153,NaN,NaN,NaN,NaN,LC,...,1,0,XY_NO,A_NO,97607750,97607751,157353,Mir_340,.,ch.X.97651759F
485510,ch.X.97737721F,97851064,97851066,97851065,97851115,NaN,NaN,NaN,NaN,LC,...,21,0,XY_NO,A_YES,97607750,97607751,243315,Mir_340,.,ch.X.97737721F


In [5]:
map_annotation= annotation[['ID', 'Closest_TSS','Closest_TSS_gene_name', 'Distance_closest_TSS']]
map_annotation

,ID,Closest_TSS,Closest_TSS_gene_name,Distance_closest_TSS
0,cg00000029,53468350,RBL2,-238
1,cg00000108,37458757,C3orf35,449
2,cg00000109,171851260,FNDC3B,64777
3,cg00000165,91182793,BARHL2,-11880
4,cg00000236,42251727,VDAC3,11567
...,...,...,...,...
485507,ch.X.97129969R,97607750,Mir_340,-364437
485508,ch.X.97133160R,97607750,Mir_340,-361246
485509,ch.X.97651759F,97607750,Mir_340,157353
485510,ch.X.97737721F,97607750,Mir_340,243315


### 2.3 Read mutation data

In [6]:
# copynumber = pd.read_csv('./UCSC-raw/Gistic2_CopyNumber_Gistic2_all_thresholded.by_genes',sep='\t')
# copynumber
mutation = pd.read_csv('./UCSC-raw/mc3.v0.2.8.PUBLIC.nonsilentGene.xena',sep='\t')
mutation

,sample,TCGA-02-0003-01,TCGA-02-0033-01,TCGA-02-0047-01,TCGA-02-0055-01,TCGA-02-2470-01,TCGA-02-2483-01,TCGA-02-2485-01,TCGA-02-2486-01,TCGA-04-1353-01,...,TCGA-ZP-A9D4-01,TCGA-ZQ-A9CR-01,TCGA-ZR-A9CJ-01,TCGA-ZS-A9CD-01,TCGA-ZS-A9CE-01,TCGA-ZS-A9CF-01,TCGA-ZS-A9CG-01,TCGA-ZT-A8OM-01,TCGA-ZU-A8S4-01,TCGA-ZX-AA5X-01
0,UBE2Q2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CHMP1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PSMA2P1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SHQ1P1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CPHL1P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40538,PTRF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40539,DIAPH2-AS1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40540,SELV,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40541,NFIX,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
original_gene_names = mutation['sample'].copy()

# Use a regular expression to remove the period and any characters following it
mutation['sample'] = mutation['sample'].str.replace(r'\..*', '', regex=True)

# Determine how many rows were changed by comparing the new values to the original ones
rows_changed = (original_gene_names != mutation['sample']).sum()

# Output the updated DataFrame and the number of rows that were changed
mutation, rows_changed

(           sample  TCGA-02-0003-01  TCGA-02-0033-01  TCGA-02-0047-01  \
 0          UBE2Q2                0                0                0   
 1          CHMP1B                0                0                0   
 2         PSMA2P1                0                0                0   
 3          SHQ1P1                0                0                0   
 4          CPHL1P                0                0                0   
 ...           ...              ...              ...              ...   
 40538        PTRF                0                0                0   
 40539  DIAPH2-AS1                0                0                0   
 40540        SELV                0                0                0   
 40541        NFIX                0                0                0   
 40542        SELP                0                0                0   
 
        TCGA-02-0055-01  TCGA-02-2470-01  TCGA-02-2483-01  TCGA-02-2485-01  \
 0                    0                0    

### 2.4 Read gene expression data, mapping information and substitue the the gene name, and import Ensembl Dataset as a standard

In [8]:
gene_expression = pd.read_csv('./UCSC-raw/tcga_RSEM_gene_fpkm', sep='\t')
gene_expression

,sample,TCGA-19-1787-01,TCGA-S9-A7J2-01,TCGA-G3-A3CH-11,TCGA-EK-A2RE-01,TCGA-44-6778-01,TCGA-F4-6854-01,TCGA-AB-2863-03,TCGA-C8-A1HL-01,TCGA-EW-A2FS-01,...,TCGA-DJ-A2QC-01,TCGA-A8-A09K-01,TCGA-61-1907-01,TCGA-IB-7885-01,TCGA-95-7947-01,TCGA-VQ-AA6F-01,TCGA-BR-8588-01,TCGA-24-2254-01,TCGA-DD-A115-01,TCGA-FV-A3I0-11
0,ENSG00000242268.2,-9.9658,-0.3940,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-1.5522,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.5479,-9.9658,-9.9658
1,ENSG00000259041.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
2,ENSG00000270112.3,-4.2934,-3.8160,-9.9658,-9.9658,-6.5064,-9.9658,-3.8160,-9.9658,-5.5735,...,-6.5064,-9.9658,-9.9658,-9.9658,-6.5064,-9.9658,-9.9658,-6.5064,-9.9658,-9.9658
3,ENSG00000167578.16,4.8324,4.1962,3.3952,3.9099,4.9031,4.1907,3.1195,4.6949,5.6023,...,5.3042,5.0414,4.7851,4.5868,5.5778,4.8294,3.7464,4.4810,3.7006,2.8720
4,ENSG00000278814.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60493,ENSG00000273233.1,-1.7322,-1.9379,-9.9658,-2.2447,-9.9658,-9.9658,-9.9658,-9.9658,-1.0559,...,-2.9324,-9.9658,-9.9658,-4.2934,-3.0469,-4.6082,-2.7274,-2.4659,-9.9658,-9.9658
60494,ENSG00000105063.18,4.7986,4.1228,3.2737,4.9393,5.2300,5.2702,6.2495,4.1261,4.3068,...,5.0153,4.8719,4.1211,4.4906,5.4397,5.6727,4.9341,5.6645,4.1400,3.0002
60495,ENSG00000231119.2,-0.6873,-4.0350,-2.7274,-5.5735,-1.7809,-9.9658,-1.3921,-5.0116,-1.9942,...,1.5612,-3.1714,-0.2671,-5.0116,-2.3147,-2.1140,-3.8160,-1.1488,-2.7274,-3.3076
60496,ENSG00000280861.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [9]:
gene_expression_map = pd.read_csv('./UCSC-raw/probeMap_gencode.v23.annotation.gene.probemap', sep='\t')

In [10]:
gene_expression_map

,id,gene,chrom,chromStart,chromEnd,strand
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,+
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,-
2,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,-
3,ENSG00000243485.3,RP11-34P13.3,chr1,29554,31109,+
4,ENSG00000274890.1,MIR1302-2,chr1,30366,30503,+
...,...,...,...,...,...,...
60493,ENSG00000198695.2,MT-ND6,chrM,14149,14673,-
60494,ENSG00000210194.1,MT-TE,chrM,14674,14742,-
60495,ENSG00000198727.2,MT-CYB,chrM,14747,15887,+
60496,ENSG00000210195.2,MT-TT,chrM,15888,15953,+


In [11]:
expression_merged = pd.merge(gene_expression, gene_expression_map, left_on='sample', right_on='id', how='left')

In [12]:
expression_merged.drop(columns=['sample','chrom', 'chromStart','chromEnd','strand','id'], inplace=True)

In [13]:
# set gene to the first column
cols = ['gene'] + [col for col in expression_merged if col != 'gene']
gene_expression = expression_merged[cols]

In [14]:
gene_expression 

,gene,TCGA-19-1787-01,TCGA-S9-A7J2-01,TCGA-G3-A3CH-11,TCGA-EK-A2RE-01,TCGA-44-6778-01,TCGA-F4-6854-01,TCGA-AB-2863-03,TCGA-C8-A1HL-01,TCGA-EW-A2FS-01,...,TCGA-DJ-A2QC-01,TCGA-A8-A09K-01,TCGA-61-1907-01,TCGA-IB-7885-01,TCGA-95-7947-01,TCGA-VQ-AA6F-01,TCGA-BR-8588-01,TCGA-24-2254-01,TCGA-DD-A115-01,TCGA-FV-A3I0-11
0,RP11-368I23.2,-9.9658,-0.3940,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-1.5522,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.5479,-9.9658,-9.9658
1,RP11-167B3.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
2,RP11-742D12.2,-4.2934,-3.8160,-9.9658,-9.9658,-6.5064,-9.9658,-3.8160,-9.9658,-5.5735,...,-6.5064,-9.9658,-9.9658,-9.9658,-6.5064,-9.9658,-9.9658,-6.5064,-9.9658,-9.9658
3,RAB4B,4.8324,4.1962,3.3952,3.9099,4.9031,4.1907,3.1195,4.6949,5.6023,...,5.3042,5.0414,4.7851,4.5868,5.5778,4.8294,3.7464,4.4810,3.7006,2.8720
4,AC104071.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60493,RP11-713D19.1,-1.7322,-1.9379,-9.9658,-2.2447,-9.9658,-9.9658,-9.9658,-9.9658,-1.0559,...,-2.9324,-9.9658,-9.9658,-4.2934,-3.0469,-4.6082,-2.7274,-2.4659,-9.9658,-9.9658
60494,PPP6R1,4.7986,4.1228,3.2737,4.9393,5.2300,5.2702,6.2495,4.1261,4.3068,...,5.0153,4.8719,4.1211,4.4906,5.4397,5.6727,4.9341,5.6645,4.1400,3.0002
60495,RP4-569M23.2,-0.6873,-4.0350,-2.7274,-5.5735,-1.7809,-9.9658,-1.3921,-5.0116,-1.9942,...,1.5612,-3.1714,-0.2671,-5.0116,-2.3147,-2.1140,-3.8160,-1.1488,-2.7274,-3.3076
60496,AC009260.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [15]:
original_gene_names = gene_expression['gene'].copy()

# Use a regular expression to remove the period and any characters following it
gene_expression['gene'] = gene_expression['gene'].str.replace(r'\..*', '', regex=True)

# Determine how many rows were changed by comparing the new values to the original ones
rows_changed = (original_gene_names != gene_expression['gene']).sum()

# Output the updated DataFrame and the number of rows that were changed
gene_expression, rows_changed

(              gene  TCGA-19-1787-01  TCGA-S9-A7J2-01  TCGA-G3-A3CH-11  \
 0      RP11-368I23          -9.9658          -0.3940          -9.9658   
 1       RP11-167B3          -9.9658          -9.9658          -9.9658   
 2      RP11-742D12          -4.2934          -3.8160          -9.9658   
 3            RAB4B           4.8324           4.1962           3.3952   
 4         AC104071          -9.9658          -9.9658          -9.9658   
 ...            ...              ...              ...              ...   
 60493  RP11-713D19          -1.7322          -1.9379          -9.9658   
 60494       PPP6R1           4.7986           4.1228           3.2737   
 60495   RP4-569M23          -0.6873          -4.0350          -2.7274   
 60496     AC009260          -9.9658          -9.9658          -9.9658   
 60497        OR8D4          -9.9658          -9.9658          -9.9658   
 
        TCGA-EK-A2RE-01  TCGA-44-6778-01  TCGA-F4-6854-01  TCGA-AB-2863-03  \
 0              -9.9658         

In [17]:
# this will be used in part4
ensembl_data_unique_gene = pd.read_csv("./UCSC-raw/Meta-Data/mart_export_unique_gene.txt")
ensembl_data_unique_gene

,Gene name
0,MT-TF
1,MT-RNR1
2,MT-TV
3,MT-RNR2
4,MT-TL1
...,...
48105,ZNF593
48106,CNKSR1
48107,ZPLD2P
48108,DDX11L2


### 2.5 Read clinical data

In [18]:
survival = pd.read_csv('./UCSC-raw/Survival_SupplementalTable_S1_20171025_xena_sp', sep='\t')

In [19]:
survival

,sample,_PATIENT,cancer type abbreviation,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,residual_tumor,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction
0,TCGA-OR-A5J1-01,TCGA-OR-A5J1,ACC,58.0,MALE,WHITE,Stage II,NaN,Adrenocortical carcinoma- Usual Type,NaN,...,NaN,1.0,1355.0,1.0,1355.0,1.0,754.0,1.0,754.0,NaN
1,TCGA-OR-A5J2-01,TCGA-OR-A5J2,ACC,44.0,FEMALE,WHITE,Stage IV,NaN,Adrenocortical carcinoma- Usual Type,NaN,...,NaN,1.0,1677.0,1.0,1677.0,NaN,NaN,1.0,289.0,NaN
2,TCGA-OR-A5J3-01,TCGA-OR-A5J3,ACC,23.0,FEMALE,WHITE,Stage III,NaN,Adrenocortical carcinoma- Usual Type,NaN,...,NaN,0.0,2091.0,0.0,2091.0,1.0,53.0,1.0,53.0,NaN
3,TCGA-OR-A5J4-01,TCGA-OR-A5J4,ACC,23.0,FEMALE,WHITE,Stage IV,NaN,Adrenocortical carcinoma- Usual Type,NaN,...,NaN,1.0,423.0,1.0,423.0,NaN,NaN,1.0,126.0,NaN
4,TCGA-OR-A5J5-01,TCGA-OR-A5J5,ACC,30.0,MALE,WHITE,Stage III,NaN,Adrenocortical carcinoma- Usual Type,NaN,...,NaN,1.0,365.0,1.0,365.0,NaN,NaN,1.0,50.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12586,TCGA-YZ-A980-01,TCGA-YZ-A980,UVM,75.0,MALE,WHITE,Stage IIIA,Stage IIIA,Spindle Cell|Epithelioid Cell,NaN,...,NaN,0.0,1862.0,0.0,1862.0,NaN,NaN,1.0,1556.0,NaN
12587,TCGA-YZ-A982-01,TCGA-YZ-A982,UVM,79.0,FEMALE,WHITE,Stage IIIB,Stage IIIB,Spindle Cell,NaN,...,NaN,0.0,495.0,0.0,495.0,NaN,NaN,0.0,495.0,NaN
12588,TCGA-YZ-A983-01,TCGA-YZ-A983,UVM,51.0,FEMALE,WHITE,Stage IIB,Stage IIB,Epithelioid Cell,NaN,...,NaN,0.0,798.0,0.0,798.0,NaN,NaN,0.0,798.0,NaN
12589,TCGA-YZ-A984-01,TCGA-YZ-A984,UVM,50.0,FEMALE,WHITE,Stage IIB,Stage IIIA,Spindle Cell|Epithelioid Cell,NaN,...,NaN,1.0,1396.0,1.0,1396.0,NaN,NaN,1.0,154.0,NaN


### 2.6 Read immune subtype data

In [20]:
immune_subtype = pd.read_csv('./UCSC-raw/Subtype_Immune_Model_Based.txt',sep='\t')

In [21]:
immune_subtype

,sample,Subtype_Immune_Model_Based
0,TCGA-A5-A0GI-01,Wound Healing (Immune C1)
1,TCGA-S9-A7J2-01,Immunologically Quiet (Immune C5)
2,TCGA-EK-A2RE-01,IFN-gamma Dominant (Immune C2)
3,TCGA-D5-5538-01,IFN-gamma Dominant (Immune C2)
4,TCGA-F4-6854-01,Wound Healing (Immune C1)
...,...,...
9121,TCGA-IB-7885-01,Wound Healing (Immune C1)
9122,TCGA-B6-A0IA-01,Lymphocyte Depleted (Immune C4)
9123,TCGA-VQ-AA6F-01,IFN-gamma Dominant (Immune C2)
9124,TCGA-BR-8588-01,IFN-gamma Dominant (Immune C2)


### 2.7 Read proteomics data

In [22]:
protein = pd.read_csv('./UCSC-raw/TCGA-RPPA-pancan-clean.xena',sep='\t')

In [23]:
protein

,SampleID,TCGA-FI-A2EY-01,TCGA-DF-A2KS-01,TCGA-A5-A1OH-01,TCGA-AX-A2H7-01,TCGA-AX-A2HA-01,TCGA-A5-A2K4-01,TCGA-EY-A2OM-01,TCGA-EY-A2ON-01,TCGA-E6-A2P8-01,...,TCGA-RD-A8N0-01,TCGA-RD-A8MW-01,TCGA-VQ-A8E2-01,TCGA-VQ-A8E0-01,TCGA-VQ-A8DL-01,TCGA-VQ-A8DU-01,TCGA-VQ-A8DT-01,TCGA-IN-A7NR-01,TCGA-RD-A8MV-01,TCGA-KB-A93G-01
0,X1433EPSILON,-0.013829,-0.168630,0.038842,0.021308,0.108640,-0.241980,0.051101,0.183180,-0.062962,...,-0.30092,-0.006139,-0.022418,0.001432,0.027505,-0.048516,-0.13928,-0.23253,-0.019379,-0.064041
1,X4EBP1,-1.127400,0.165870,-0.382370,-0.717660,0.090459,-0.378660,-0.044557,-0.053956,0.008223,...,-0.48837,-0.321200,-0.386970,0.018423,-0.325220,-0.073244,-0.16909,-0.50985,0.250550,-0.247550
2,X4EBP1_pS65,-0.423550,-0.505950,0.042306,-0.493150,0.339620,-0.055426,-0.370140,0.053458,0.268220,...,-0.24151,-0.098391,-0.044874,-0.064016,0.000648,-0.034748,0.12588,0.42930,0.376320,-0.042583
3,X4EBP1_pT37T46,-0.827380,0.019504,0.119400,-0.370670,-0.017032,0.231690,-0.041726,-0.185100,-0.741640,...,0.29797,-0.597470,-0.858290,-1.465400,-0.539250,-0.649850,-1.21630,0.44429,0.926290,-0.014209
4,X53BP1,0.021656,-0.947890,0.374310,0.195980,0.266380,-0.171180,0.755470,-0.650250,-0.237940,...,-0.47680,-1.428200,-0.330090,-2.059200,-1.222300,-0.407540,-0.65722,-0.56508,-2.084400,-1.377900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,NAPSINA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,P63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,RET_pY905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,SYNAPTOPHYSIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.8 Read molecular subtype

In [24]:
cellsub = pd.read_csv('./UCSC-raw/TCGASubtype.20170308.tsv', sep='\t')

In [25]:
cellsub

,sampleID,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected
0,TCGA-OR-A5J1-01,steroid-phenotype-high+proliferation,CIMP-high,NaN,miRNA_1,Quiet,COC3,C1A,ACC.CIMP-high
1,TCGA-OR-A5J2-01,steroid-phenotype-high+proliferation,CIMP-low,1,miRNA_1,Noisy,COC3,C1A,ACC.CIMP-low
2,TCGA-OR-A5J3-01,steroid-phenotype-high,CIMP-intermediate,3,miRNA_6,Chromosomal,COC2,C1A,ACC.CIMP-intermediate
3,TCGA-OR-A5J4-01,NaN,CIMP-high,NaN,miRNA_6,Chromosomal,NaN,NaN,ACC.CIMP-high
4,TCGA-OR-A5J5-01,steroid-phenotype-high,CIMP-intermediate,NaN,miRNA_2,Chromosomal,COC2,C1A,ACC.CIMP-intermediate
...,...,...,...,...,...,...,...,...,...
7729,TCGA-ND-A4WA-01,2,NaN,NaN,NaN,NaN,NaN,NaN,UCS.2
7730,TCGA-NG-A4VU-01,1,NaN,NaN,NaN,NaN,NaN,NaN,UCS.1
7731,TCGA-QN-A5NN-01,2,NaN,NaN,NaN,NaN,NaN,NaN,UCS.2
7732,TCGA-NF-A5CP-01,2,NaN,NaN,NaN,NaN,NaN,NaN,UCS.2


### 2.9 Read sample type and primary disease

In [26]:
dense = pd.read_csv('./UCSC-raw/TCGA_phenotype_denseDataOnlyDownload.tsv', sep='\t')

In [27]:
dense

,sample,sample_type_id,sample_type,_primary_disease
0,TCGA-D3-A1QA-07,7.0,Additional Metastatic,skin cutaneous melanoma
1,TCGA-DE-A4MD-06,6.0,Metastatic,thyroid carcinoma
2,TCGA-J8-A3O2-06,6.0,Metastatic,thyroid carcinoma
3,TCGA-J8-A3YH-06,6.0,Metastatic,thyroid carcinoma
4,TCGA-EM-A2P1-06,6.0,Metastatic,thyroid carcinoma
...,...,...,...,...
12799,TCGA-17-Z059-01,NaN,NaN,lung adenocarcinoma
12800,TCGA-17-Z060-01,NaN,NaN,lung adenocarcinoma
12801,TCGA-17-Z061-01,NaN,NaN,lung adenocarcinoma
12802,TCGA-17-Z062-01,NaN,NaN,lung adenocarcinoma


### 2.10 Unify patient samples within methylation, copynumer,  gene expression, clinical, proteomics, molecular subtype, and sample type, primary disease datasets

In [28]:
# molecular subtype
cellsub.rename(columns={'sampleID': 'sample'}, inplace=True)
cellsub

,sample,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected
0,TCGA-OR-A5J1-01,steroid-phenotype-high+proliferation,CIMP-high,NaN,miRNA_1,Quiet,COC3,C1A,ACC.CIMP-high
1,TCGA-OR-A5J2-01,steroid-phenotype-high+proliferation,CIMP-low,1,miRNA_1,Noisy,COC3,C1A,ACC.CIMP-low
2,TCGA-OR-A5J3-01,steroid-phenotype-high,CIMP-intermediate,3,miRNA_6,Chromosomal,COC2,C1A,ACC.CIMP-intermediate
3,TCGA-OR-A5J4-01,NaN,CIMP-high,NaN,miRNA_6,Chromosomal,NaN,NaN,ACC.CIMP-high
4,TCGA-OR-A5J5-01,steroid-phenotype-high,CIMP-intermediate,NaN,miRNA_2,Chromosomal,COC2,C1A,ACC.CIMP-intermediate
...,...,...,...,...,...,...,...,...,...
7729,TCGA-ND-A4WA-01,2,NaN,NaN,NaN,NaN,NaN,NaN,UCS.2
7730,TCGA-NG-A4VU-01,1,NaN,NaN,NaN,NaN,NaN,NaN,UCS.1
7731,TCGA-QN-A5NN-01,2,NaN,NaN,NaN,NaN,NaN,NaN,UCS.2
7732,TCGA-NF-A5CP-01,2,NaN,NaN,NaN,NaN,NaN,NaN,UCS.2


In [29]:
# Upstream_df_filtered, Distal_Promoter_df_filtered, Proximal_Promoter_df_filtered, Core_Promoter_df_filtered, Downstream_df_filtered
# mutation_filtered, gene_expression_filtered
# survival, protein, cellsub, dense

# Extract column names starting with 'TCGA' from methylation datasets
tcga_columns_methylation = [col for col in methylation_value.columns if col.startswith('TCGA')]

# Extract 'TCGA' columns from other datasets
tcga_columns_mutation = [col for col in mutation.columns if col.startswith('TCGA')]
tcga_columns_gene_expression = [col for col in gene_expression.columns if col.startswith('TCGA')]
tcga_columns_survival = [col for col in survival['sample'] if col.startswith('TCGA')]
tcga_columns_protein = [col for col in protein.columns if col.startswith('TCGA')]
tcga_columns_cellsub = [col for col in cellsub['sample'] if col.startswith('TCGA')]
tcga_columns_dense = [col for col in dense['sample'] if col.startswith('TCGA')]
tcga_columns_immune_subtype = [col for col in immune_subtype['sample'] if col.startswith('TCGA')]
# Find the intersection of TCGA column names across all DataFrames
common_tcga_columns = set(tcga_columns_methylation) & set(tcga_columns_mutation) & set(tcga_columns_gene_expression) & set(tcga_columns_survival) & set(tcga_columns_protein) & set(tcga_columns_cellsub) & set(tcga_columns_dense) & set(tcga_columns_immune_subtype)

# Convert the intersection back to a list, if needed
common_tcga_columns_list = sorted(list(common_tcga_columns))

# Print the number and the list of common TCGA columns
print(f"Number of common TCGA columns: {len(common_tcga_columns)}")

Number of common TCGA columns: 3592


In [30]:
# Define columns to keep along with common TCGA columns
additional_cols_methylation = ['sample']

# Filter each methylation DataFrame
methylation_value = methylation_value[additional_cols_methylation + common_tcga_columns_list]


In [31]:
methylation_value

,sample,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,cg00000029,0.271459,0.178549,0.421797,0.356337,0.305986,0.213924,0.154276,0.123989,0.632777,...,0.220225,0.188538,0.399458,0.230607,0.077412,0.213295,0.147800,0.209075,0.424846,0.125621
1,cg00000165,0.237847,0.769536,0.572464,0.496236,0.493429,0.271637,0.667137,0.605534,0.855148,...,0.708786,0.405914,0.555105,0.345176,0.073410,0.186770,0.213624,0.083344,0.512164,0.472287
2,cg00000236,0.819380,0.832079,0.829525,0.841864,0.871185,0.805841,0.839183,0.818799,0.903875,...,0.823639,0.871951,0.872767,0.829245,0.896582,0.892500,0.858856,0.840979,0.884508,0.847884
3,cg00000289,0.581186,0.664975,0.553852,0.625890,0.662169,0.588728,0.652469,0.642547,0.672771,...,0.495494,0.582348,0.675632,0.511649,0.779501,0.715340,0.694251,0.789815,0.788009,0.753984
4,cg00000292,0.695524,0.667007,0.696069,0.651084,0.711111,0.780729,0.692352,0.682607,0.673339,...,0.598891,0.638896,0.582151,0.852534,0.433840,0.699675,0.612998,0.633664,0.744623,0.684432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396060,rs9363764,0.546181,0.085512,0.431087,0.445115,0.590979,0.825421,0.330264,0.885588,0.516052,...,0.079002,0.047067,0.947840,0.868085,0.546954,0.044764,0.564664,0.952047,0.580652,0.569608
396061,rs939290,0.543712,0.914583,0.052077,0.935518,0.546602,0.733640,0.522601,0.917549,0.530067,...,0.943839,0.787211,0.964255,0.058715,0.025949,0.025922,0.032900,0.513310,0.030066,0.973873
396062,rs951295,0.539201,0.927806,0.557405,0.066231,0.632937,0.949273,0.060050,0.058859,0.529061,...,0.328963,0.034411,0.545621,0.956065,0.543114,0.027296,0.510900,0.970230,0.032954,0.030402
396063,rs966367,0.445020,0.474664,0.060683,0.052615,0.550181,0.069499,0.474534,0.484516,0.947635,...,0.049456,0.028963,0.038205,0.067768,0.505955,0.026497,0.452829,0.027822,0.034293,0.030099


In [32]:
# Define columns to keep along with common TCGA columns
additional_cols_mutation = ['sample']

# Filter the mutation DataFrame
mutation = mutation[additional_cols_mutation + common_tcga_columns_list]
mutation

,sample,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,UBE2Q2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CHMP1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PSMA2P1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SHQ1P1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CPHL1P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40538,PTRF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40539,DIAPH2-AS1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40540,SELV,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40541,NFIX,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# Define columns to keep along with common TCGA columns
additional_cols_gene_expression = ['gene']

# Filter the gene expression DataFrame
gene_expression = gene_expression[additional_cols_gene_expression + common_tcga_columns_list]
gene_expression

,gene,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,RP11-368I23,-3.1714,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-1.6394,...,-3.0469,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
1,RP11-167B3,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
2,RP11-742D12,-9.9658,-6.5064,-6.5064,-9.9658,-9.9658,-9.9658,-9.9658,-6.5064,-6.5064,...,-5.5735,-9.9658,-9.9658,-6.5064,-9.9658,-9.9658,-9.9658,-2.7274,-9.9658,-9.9658
3,RAB4B,5.2064,4.0100,4.6095,5.2238,4.8223,5.8136,4.7704,4.4829,5.2721,...,4.4894,4.0225,4.4602,5.5059,4.9718,4.1986,5.0769,4.8028,4.3924,4.4470
4,AC104071,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60493,RP11-713D19,-9.9658,-2.7274,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-2.9324,-2.1779,...,-1.5951,-3.4580,-9.9658,-9.9658,-2.9324,-2.5479,-9.9658,-9.9658,-3.8160,-9.9658
60494,PPP6R1,4.9393,4.2795,4.5405,4.7280,4.5454,4.0162,4.7469,4.4390,5.0122,...,5.5601,5.1768,4.9472,4.4983,4.9644,4.3177,4.6399,3.9636,3.8521,4.2274
60495,RP4-569M23,-1.0262,-1.9942,-1.4699,-2.8262,-3.1714,0.5170,-5.5735,-4.0350,-3.6259,...,-2.3147,-2.3884,-2.6349,-1.6394,-3.6259,-2.6349,-2.5479,-2.1779,-2.9324,-3.0469
60496,AC009260,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [34]:
# Define columns to keep along with common TCGA columns
additional_cols_protein = ['SampleID']

# Filter the protein DataFrame
protein = protein[additional_cols_protein + common_tcga_columns_list]
protein

,SampleID,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,X1433EPSILON,-0.14547,-0.219520,-0.333630,-0.277350,-0.152610,-0.15745,-0.238240,-0.085358,0.34770,...,-0.036610,0.16146,-0.042626,-0.024017,0.097818,-0.004645,0.109000,0.163030,0.136930,0.10366
1,X4EBP1,-0.88448,-0.568390,-0.657050,-0.837460,0.113180,-0.84132,-0.550910,0.980820,-0.51069,...,0.268470,0.70671,-0.139000,0.082556,0.001678,-0.160760,0.091391,-0.076903,0.401560,0.14597
2,X4EBP1_pS65,-0.23857,0.190620,-0.289490,-0.054096,0.055524,-0.35460,-0.146360,0.507140,0.33609,...,-0.072575,-0.30366,-0.067930,-0.477660,-0.340900,-0.283850,0.110350,0.252940,-0.083075,0.31218
3,X4EBP1_pT37T46,0.34236,0.623000,0.391190,0.312460,-0.154030,-0.35322,-0.413680,-0.703410,1.13080,...,-0.393010,0.16711,-0.175750,-0.335890,-0.200440,0.321750,0.370270,0.568820,-0.066879,0.90364
4,X53BP1,-0.26811,0.637540,0.241270,-0.572200,-0.446210,-0.23335,-0.362280,0.202410,0.51428,...,0.183590,-0.71555,-1.295700,-0.085562,0.114300,-1.945800,-0.418910,-1.449800,-0.965190,-0.82798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,NAPSINA,0.32483,1.176700,0.450220,0.525790,0.566730,2.14920,-0.080169,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,P63,0.94524,1.766900,1.119600,1.090600,-0.170610,0.41600,0.316990,-1.361800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,RET_pY905,0.27270,-0.099365,0.036639,0.509100,-0.244330,0.35593,-0.124720,-0.468070,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,SYNAPTOPHYSIN,0.64063,0.985540,0.656660,0.175850,1.803600,0.56185,0.159050,-0.454340,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Filter rows based on common TCGA identifiers
immune_subtype_filtered = immune_subtype[immune_subtype['sample'].isin(common_tcga_columns_list)]
immune_subtype_filtered = immune_subtype_filtered.sort_values(by=['sample']).reset_index(drop=True)

survival_filtered = survival[survival['sample'].isin(common_tcga_columns_list)]
survival_filtered = survival_filtered.sort_values(by=['sample']).reset_index(drop=True)

cellsub.rename(columns={'sampleID': 'sample'}, inplace=True)
cellsub_filtered = cellsub[cellsub['sample'].isin(common_tcga_columns_list)]
cellsub_filtered = cellsub_filtered.sort_values(by=['sample']).reset_index(drop=True)

dense_filtered = dense[dense['sample'].isin(common_tcga_columns_list)]
dense_filtered = dense_filtered.sort_values(by=['sample']).reset_index(drop=True)

In [36]:
immune_subtype_filtered

,sample,Subtype_Immune_Model_Based
0,TCGA-05-4384-01,Inflammatory (Immune C3)
1,TCGA-05-4396-01,Inflammatory (Immune C3)
2,TCGA-05-4405-01,Inflammatory (Immune C3)
3,TCGA-05-4410-01,Inflammatory (Immune C3)
4,TCGA-05-4417-01,Inflammatory (Immune C3)
...,...,...
3587,TCGA-ZA-A8F6-01,Inflammatory (Immune C3)
3588,TCGA-ZG-A8QW-01,Inflammatory (Immune C3)
3589,TCGA-ZG-A8QX-01,Inflammatory (Immune C3)
3590,TCGA-ZG-A8QY-01,Inflammatory (Immune C3)


## 3.Methylation data process

### 3.1 Define methylation region

In [37]:
import pandas as pd
import numpy as np

#Define vectorized area determination function for methylation data
def vectorized_determine_region(distances):
    regions = ['Upstream', 'Distal Promoter', 'Proximal Promoter', 'Core Promoter', 'Downstream']
    conditions = [
        (-6000 <= distances) & (distances < -3000),
        (-3000 <= distances) & (distances < -250),
        (-250 <= distances) & (distances < -50),
        (-50 <= distances) & (distances <= 0),
        (0 < distances) & (distances <= 3000)
    ]
    return np.select(conditions, regions, default=None)

### 3.2 Merge the annotation files to the methylation data and apply region function

In [38]:
# Merging basic data and methylation data
methylation_merged_df = pd.merge(map_annotation, methylation_value, left_on='ID', right_on='sample', how='right')

# Determining the region for each row outside the loop
methylation_merged_df['Region'] = vectorized_determine_region(methylation_merged_df['Distance_closest_TSS'])

methylation_merged_df = methylation_merged_df.dropna(subset=['Region'])  # Remove rows without a region

# Initializing a dictionary to store data for each region
regions_data = {region: pd.DataFrame() for region in ["Upstream", "Distal Promoter", "Proximal Promoter", "Core Promoter", "Downstream"]}

In [39]:
methylation_merged_df

,ID,Closest_TSS,Closest_TSS_gene_name,Distance_closest_TSS,sample,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,...,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01,Region
0,cg00000029,53468350,RBL2,-238.0,cg00000029,0.271459,0.178549,0.421797,0.356337,0.305986,...,0.188538,0.399458,0.230607,0.077412,0.213295,0.147800,0.209075,0.424846,0.125621,Proximal Promoter
4,cg00000292,28889808,ATP2A1,292.0,cg00000292,0.695524,0.667007,0.696069,0.651084,0.711111,...,0.638896,0.582151,0.852534,0.433840,0.699675,0.612998,0.633664,0.744623,0.684432,Downstream
5,cg00000321,41166989,SFRP1,-812.0,cg00000321,0.458521,0.189516,0.582229,0.484243,0.364884,...,0.478930,0.371265,0.501064,0.131394,0.379000,0.581186,0.578728,0.587923,0.533312,Distal Promoter
7,cg00000622,23034426,NIPA2,-20.0,cg00000622,0.042373,0.041281,0.032737,0.009248,0.013139,...,0.012708,0.013785,0.015913,0.011497,0.015795,0.015224,0.013677,0.012833,0.012404,Core Promoter
8,cg00000658,139997799,MAN1B1,125.0,cg00000658,0.756279,0.842386,0.811916,0.812574,0.829621,...,0.922465,0.812436,0.912698,0.887378,0.854306,0.880414,0.844765,0.861372,0.845371,Downstream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395958,ch.9.98959675F,99916185,DQ579969,-3668.0,ch.9.98959675F,0.177846,0.199775,0.114339,0.071666,0.175671,...,0.184215,0.095399,NaN,0.050152,0.145695,0.131008,0.109962,0.152262,0.129312,Upstream
395965,ch.X.1084981F,70679401,TAF1,1072.0,ch.X.1084981F,0.066781,0.112509,0.075506,0.068982,0.068746,...,0.031316,0.039315,0.035488,0.024255,0.028416,0.037402,0.042238,0.040425,0.043973,Downstream
395974,ch.X.1776338R,122762640,Mir_584,-818.0,ch.X.1776338R,0.118205,0.108565,0.077662,0.052337,0.051421,...,0.105029,0.087976,0.090018,0.033905,0.078914,0.118443,0.087562,0.127812,0.103788,Distal Promoter
395976,ch.X.188238R,12817325,PRPS2,-3783.0,ch.X.188238R,0.152843,0.099112,0.055423,0.038722,0.055517,...,0.272626,0.096108,0.115536,0.069752,0.060663,0.087765,0.059504,0.106371,0.101929,Upstream


In [40]:
# Delete the 'sample' column
methylation_merged_df = methylation_merged_df.drop('sample', axis=1)
# Delete the 'ID' column
methylation_merged_df = methylation_merged_df.drop('ID', axis=1)
# Delete the 'Distance_closest_TSS' column
methylation_merged_df = methylation_merged_df.drop('Distance_closest_TSS', axis=1)


In [41]:
methylation_merged_df

,Closest_TSS,Closest_TSS_gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,...,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01,Region
0,53468350,RBL2,0.271459,0.178549,0.421797,0.356337,0.305986,0.213924,0.154276,0.123989,...,0.188538,0.399458,0.230607,0.077412,0.213295,0.147800,0.209075,0.424846,0.125621,Proximal Promoter
4,28889808,ATP2A1,0.695524,0.667007,0.696069,0.651084,0.711111,0.780729,0.692352,0.682607,...,0.638896,0.582151,0.852534,0.433840,0.699675,0.612998,0.633664,0.744623,0.684432,Downstream
5,41166989,SFRP1,0.458521,0.189516,0.582229,0.484243,0.364884,0.278198,0.525176,0.264286,...,0.478930,0.371265,0.501064,0.131394,0.379000,0.581186,0.578728,0.587923,0.533312,Distal Promoter
7,23034426,NIPA2,0.042373,0.041281,0.032737,0.009248,0.013139,0.042395,0.024518,0.021457,...,0.012708,0.013785,0.015913,0.011497,0.015795,0.015224,0.013677,0.012833,0.012404,Core Promoter
8,139997799,MAN1B1,0.756279,0.842386,0.811916,0.812574,0.829621,0.816594,0.818160,0.798735,...,0.922465,0.812436,0.912698,0.887378,0.854306,0.880414,0.844765,0.861372,0.845371,Downstream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395958,99916185,DQ579969,0.177846,0.199775,0.114339,0.071666,0.175671,NaN,0.051075,0.143281,...,0.184215,0.095399,NaN,0.050152,0.145695,0.131008,0.109962,0.152262,0.129312,Upstream
395965,70679401,TAF1,0.066781,0.112509,0.075506,0.068982,0.068746,0.095323,0.035396,0.049945,...,0.031316,0.039315,0.035488,0.024255,0.028416,0.037402,0.042238,0.040425,0.043973,Downstream
395974,122762640,Mir_584,0.118205,0.108565,0.077662,0.052337,0.051421,0.076293,0.077988,0.043371,...,0.105029,0.087976,0.090018,0.033905,0.078914,0.118443,0.087562,0.127812,0.103788,Distal Promoter
395976,12817325,PRPS2,0.152843,0.099112,0.055423,0.038722,0.055517,0.084918,0.070714,0.030774,...,0.272626,0.096108,0.115536,0.069752,0.060663,0.087765,0.059504,0.106371,0.101929,Upstream


In [42]:
original_gene_names = methylation_merged_df['Closest_TSS_gene_name'].copy()

# Use a regular expression to remove the period and any characters following it
methylation_merged_df['Closest_TSS_gene_name'] = methylation_merged_df['Closest_TSS_gene_name'].str.replace(r'\..*', '', regex=True)

# Determine how many rows were changed by comparing the new values to the original ones
rows_changed = (original_gene_names != methylation_merged_df['Closest_TSS_gene_name']).sum()

# Output the updated DataFrame and the number of rows that were changed
methylation_merged_df, rows_changed

(       Closest_TSS Closest_TSS_gene_name  TCGA-05-4384-01  TCGA-05-4396-01  \
 0         53468350                  RBL2         0.271459         0.178549   
 4         28889808                ATP2A1         0.695524         0.667007   
 5         41166989                 SFRP1         0.458521         0.189516   
 7         23034426                 NIPA2         0.042373         0.041281   
 8        139997799                MAN1B1         0.756279         0.842386   
 ...            ...                   ...              ...              ...   
 395958    99916185              DQ579969         0.177846         0.199775   
 395965    70679401                  TAF1         0.066781         0.112509   
 395974   122762640               Mir_584         0.118205         0.108565   
 395976    12817325                 PRPS2         0.152843         0.099112   
 395993    47064246                  INE1         0.102478         0.065977   
 
         TCGA-05-4405-01  TCGA-05-4410-01  TCGA-05

In [43]:
methylation_merged_df['Closest_TSS'] = methylation_merged_df['Closest_TSS'].astype(int)
methylation_merged_df['Closest_TSS_gene_name'] = methylation_merged_df['Closest_TSS_gene_name'].astype(str)
methylation_merged_df['Region'] = methylation_merged_df['Region'].astype(str)

In [44]:
print(methylation_merged_df[['Closest_TSS', 'Closest_TSS_gene_name', 'Region']].dtypes)

Closest_TSS               int32
Closest_TSS_gene_name    object
Region                   object
dtype: object


### 3.3 Calculate the average methylation value of five regions

In [45]:
# find columns not started with 'TCGA'
non_tcga_columns = methylation_merged_df.filter(regex='^(?!TCGA)').columns

print("Columns not starting with TCGA:")
print(non_tcga_columns)

Columns not starting with TCGA:
Index(['Closest_TSS', 'Closest_TSS_gene_name', 'Region'], dtype='object')


In [46]:
# obtain all regions
regions = methylation_merged_df['Region'].unique()
regions

array(['Proximal Promoter', 'Downstream', 'Distal Promoter',
       'Core Promoter', 'Upstream'], dtype=object)

In [47]:
import pandas as pd
# Initialize empty DataFrames for each region
Upstream_df = pd.DataFrame()
Distal_Promoter_df = pd.DataFrame()
Proximal_Promoter_df = pd.DataFrame()
Core_Promoter_df = pd.DataFrame()
Downstream_df = pd.DataFrame()

# Operate on each region
for region in regions:
    # Get all data for this region
    region_data = methylation_merged_df[methylation_merged_df['Region'] == region]
    
    # Group and calculate the average for each (TSS, Region) combination
    grouped = region_data.groupby(['Closest_TSS_gene_name', 'Region'], as_index=False).mean()
    
    # Since we split the data into different files based on Region, we can delete this column
    grouped = grouped.drop(columns=['Region'])
    
    # Print the shape of the grouped data
    print(f"Shape of {region}: {grouped.shape}")
    
    # Assign the grouped data to the respective DataFrame
    if region == 'Upstream':
        Upstream_df = grouped
    elif region == 'Distal Promoter':
        Distal_Promoter_df = grouped
    elif region == 'Proximal Promoter':
        Proximal_Promoter_df = grouped
    elif region == 'Core Promoter':
        Core_Promoter_df = grouped
    elif region == 'Downstream':
        Downstream_df = grouped

    # Optionally, save the data for this region to a new csv file
    # grouped.to_csv(f"{region}_averaged_tss_data.csv", index=False)


Shape of Proximal Promoter: (16169, 3594)
Shape of Downstream: (21757, 3594)
Shape of Distal Promoter: (17797, 3594)
Shape of Core Promoter: (10139, 3594)
Shape of Upstream: (5638, 3594)


### 3.4 Unify gene and TSS for five methylation value files

In [48]:
import pandas as pd

# from methylation files above DataFrame 
dfs = [Upstream_df, Distal_Promoter_df, Proximal_Promoter_df, Core_Promoter_df, Downstream_df]

# merge those files to find all combos 
all_genes_tss = pd.concat(dfs)['Closest_TSS_gene_name'].drop_duplicates()

In [49]:
all_genes_tss

0            1/2-SBSRNA4
1                  40969
2                  40972
3                  41157
4                  41158
              ...       
21670             ZNF862
21671             ZNF865
21702             ZRANB2
21705              ZRSR1
21750    hsa-miR-3194-3p
Name: Closest_TSS_gene_name, Length: 24396, dtype: object

In [50]:
# Merge unique combinations back into each DataFrame and fill NaN values with 0
# Upstream
Upstream_df = pd.merge(all_genes_tss, Upstream_df, on=['Closest_TSS_gene_name'], how='outer').fillna(0)
print(f"Shape of Upstream_df: {Upstream_df.shape}")

# Distal Promoter
Distal_Promoter_df = pd.merge(all_genes_tss, Distal_Promoter_df, on=['Closest_TSS_gene_name'], how='outer').fillna(0)
print(f"Shape of Distal_Promoter_df: {Distal_Promoter_df.shape}")

# Proximal Promoter
Proximal_Promoter_df = pd.merge(all_genes_tss, Proximal_Promoter_df, on=['Closest_TSS_gene_name'], how='outer').fillna(0)
print(f"Shape of Proximal_Promoter_df: {Proximal_Promoter_df.shape}")

# Core Promoter
Core_Promoter_df = pd.merge(all_genes_tss, Core_Promoter_df, on=['Closest_TSS_gene_name'], how='outer').fillna(0)
print(f"Shape of Core_Promoter_df: {Core_Promoter_df.shape}")

# Downstream
Downstream_df = pd.merge(all_genes_tss, Downstream_df, on=['Closest_TSS_gene_name'], how='outer').fillna(0)
print(f"Shape of Downstream_df: {Downstream_df.shape}")

Shape of Upstream_df: (24396, 3594)
Shape of Distal_Promoter_df: (24396, 3594)
Shape of Proximal_Promoter_df: (24396, 3594)
Shape of Core_Promoter_df: (24396, 3594)
Shape of Downstream_df: (24396, 3594)


## 4.Unify genes

### 4.1 Unify gene and TSS for methylation, copynumer, and gene expression data

In [51]:
mutation.rename(columns={'sample': 'gene_name'}, inplace=True)
mutation = mutation.dropna(subset=['gene_name'])
mutation = mutation.groupby('gene_name', as_index=False).mean()
mutation = mutation.sort_values(by=['gene_name']).reset_index(drop=True)
mutation

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_22064\856365089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mutation.rename(columns={'sample': 'gene_name'}, inplace=True)


,gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,A12M1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A12M2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A12M3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A12M4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40485,hsa-mir-490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40486,hsa-mir-5195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40487,hsa-mir-6080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40488,hsa-mir-7162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
gene_expression.rename(columns={'gene': 'gene_name'}, inplace=True)
gene_expression = gene_expression.dropna(subset=['gene_name'])
gene_expression = gene_expression.groupby('gene_name', as_index=False).mean()
gene_expression = gene_expression.sort_values(by=['gene_name']).reset_index(drop=True)
gene_expression

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_22064\3678210828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_expression.rename(columns={'gene': 'gene_name'}, inplace=True)


,gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,5S_rRNA,-9.965800,-9.9658,-9.9658,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,...,-7.912300,-9.168762,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,-9.965800,-9.965800
1,5_8S_rRNA,-9.965800,-9.9658,-9.9658,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,...,-9.965800,-9.965800,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,-9.965800,-9.965800
2,7SK,-9.965800,-9.9658,-9.9658,-9.965800,-9.965800,-9.965800,-8.556583,-9.9658,-7.915867,...,-6.844283,-8.977333,-9.965800,-9.965800,-9.965800,-7.988867,-9.9658,-9.965800,-9.965800,-9.965800
3,A1BG,4.169200,1.0642,4.4047,3.388300,3.490700,6.557100,3.627700,2.7205,4.557700,...,1.575800,-0.150400,1.895700,5.752800,5.086700,1.700900,2.8321,1.637400,1.202300,2.144400
4,A1BG-AS1,2.032500,-0.7346,2.3479,1.071100,1.036300,2.430600,-0.575600,0.4552,2.682800,...,-0.413100,-0.859900,-0.057400,2.698400,2.513700,-0.042500,0.2522,-1.318300,-0.968600,-0.664300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49427,snoZ40,-9.965800,-9.9658,-9.9658,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,...,-9.965800,-9.965800,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,-9.965800,-9.965800
49428,snoZ6,-9.965800,-9.9658,-9.9658,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,...,-9.965800,-9.965800,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,-9.965800,-9.965800
49429,snosnR66,-9.965800,-9.9658,-9.9658,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,...,-9.965800,-9.965800,-9.965800,-9.965800,-9.965800,-9.965800,-9.9658,-9.965800,-9.965800,-9.965800
49430,uc_338,-9.384222,-9.9658,-9.9658,-9.688533,-9.699394,-9.389294,-9.435992,-9.9658,-7.986756,...,-7.757456,-8.603053,-8.782681,-9.503536,-9.036242,-9.416556,-9.9658,-9.178531,-9.710733,-9.145553


In [53]:
protein.rename(columns={'SampleID': 'gene_name'}, inplace=True)
protein = protein.dropna(subset=['gene_name'])
protein = protein.groupby('gene_name', as_index=False).mean()
protein = protein.sort_values(by=['gene_name']).reset_index(drop=True)
protein

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_22064\3971708056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protein.rename(columns={'SampleID': 'gene_name'}, inplace=True)


,gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,ACC1,-0.16681,-0.11365,-0.102720,0.620500,0.577680,0.499920,0.095701,-0.464930,-0.19981,...,-0.090928,0.308130,0.965270,-0.033844,0.677490,-1.198200,0.495740,1.314100,1.544300,0.312930
1,ACC_pS79,-0.16784,-0.25890,-0.255800,0.479320,-0.025757,0.413260,0.074167,-0.596590,0.18365,...,-0.212270,-0.059903,0.108670,-0.048454,0.338390,-0.851730,1.120800,1.193400,1.520400,0.086799
2,ACETYLATUBULINLYS40,0.78731,1.46840,0.318170,0.659870,0.749060,0.257600,0.920650,0.333380,1.58440,...,0.331600,-1.225100,-3.450900,-0.903900,-0.000619,-1.564200,-0.222450,-0.847980,-0.535090,0.990560
3,ACVRL1,-0.10909,-0.21774,-0.267460,-0.246930,-0.220150,-0.086443,-0.005093,-0.081634,-0.20829,...,0.008964,0.269070,0.371660,-0.045248,-0.234630,0.225910,0.540900,0.444930,0.436140,0.555200
4,ADAR1,-0.16673,-0.16416,0.296280,0.073974,0.161890,0.266900,-0.051615,0.096403,-0.26779,...,0.715090,0.607130,0.323600,-0.305770,0.210320,-0.923570,-0.349430,-0.469560,-0.068306,-0.464210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,XRCC1,-0.20891,-0.31584,-0.202320,-0.093760,-0.102720,-0.268120,-0.184010,-0.315080,-0.25434,...,0.545850,-0.155830,-0.017329,0.144710,0.331000,-0.145230,0.061808,0.074207,-0.051698,0.084358
254,YAP,0.41404,0.43820,0.572040,0.211820,0.041380,0.105310,0.256950,0.885780,-0.28088,...,0.073918,-0.196840,0.021958,-0.260830,-0.480560,0.005599,0.150260,-0.117610,0.049025,-0.021126
255,YAP_pS127,0.72353,1.17790,0.935400,0.447520,-0.065946,0.353380,0.731060,0.223150,-0.58392,...,-0.522260,-0.962180,0.229290,-0.369120,-0.722460,0.129730,0.606510,0.214570,0.221440,0.629370
256,YB1,-0.72235,0.38805,0.041689,0.187200,-0.329960,-0.319810,0.137700,-0.041581,0.44919,...,-0.140280,-0.414250,-0.521560,-0.377330,0.060114,-1.070900,-0.250570,-0.285350,-0.327000,0.306040


In [54]:
# Upstream
Upstream_df.rename(columns={'Closest_TSS_gene_name': 'gene_name'}, inplace=True)
Upstream_df = Upstream_df.dropna(subset=['gene_name'])
Upstream_df = Upstream_df.groupby('gene_name', as_index=False).mean()
Upstream_df = Upstream_df.sort_values(by=['gene_name']).reset_index(drop=True)
# Find indices where 'row_name' is 'unknown'
Upstream_indices_to_drop = Upstream_df[Upstream_df['gene_name'] == 'unknown'].index
Upstream_df.drop(Upstream_indices_to_drop, inplace=True)

# Distal Promoter
Distal_Promoter_df.rename(columns={'Closest_TSS_gene_name': 'gene_name'}, inplace=True)
Distal_Promoter_df = Distal_Promoter_df.dropna(subset=['gene_name'])
Distal_Promoter_df = Distal_Promoter_df.groupby('gene_name', as_index=False).mean()
Distal_Promoter_df = Distal_Promoter_df.sort_values(by=['gene_name']).reset_index(drop=True)
# Find indices where 'row_name' is 'unknown'
Distal_Promoter_indices_to_drop = Distal_Promoter_df[Distal_Promoter_df['gene_name'] == 'unknown'].index
Distal_Promoter_df.drop(Distal_Promoter_indices_to_drop, inplace=True)

# Proximal Promoter
Proximal_Promoter_df.rename(columns={'Closest_TSS_gene_name': 'gene_name'}, inplace=True)
Proximal_Promoter_df = Proximal_Promoter_df.dropna(subset=['gene_name'])
Proximal_Promoter_df = Proximal_Promoter_df.groupby('gene_name', as_index=False).mean()
Proximal_Promoter_df = Proximal_Promoter_df.sort_values(by=['gene_name']).reset_index(drop=True)
# Find indices where 'row_name' is 'unknown'
Proximal_Promoter_indices_to_drop = Proximal_Promoter_df[Proximal_Promoter_df['gene_name'] == 'unknown'].index
Proximal_Promoter_df.drop(Proximal_Promoter_indices_to_drop, inplace=True)

# Core Promoter
Core_Promoter_df.rename(columns={'Closest_TSS_gene_name': 'gene_name'}, inplace=True)
Core_Promoter_df = Core_Promoter_df.dropna(subset=['gene_name'])
Core_Promoter_df = Core_Promoter_df.groupby('gene_name', as_index=False).mean()
Core_Promoter_df = Core_Promoter_df.sort_values(by=['gene_name']).reset_index(drop=True)
# Find indices where 'row_name' is 'unknown'
Core_Promoter_indices_to_drop = Core_Promoter_df[Core_Promoter_df['gene_name'] == 'unknown'].index
Core_Promoter_df.drop(Core_Promoter_indices_to_drop, inplace=True)

# Downstream
Downstream_df.rename(columns={'Closest_TSS_gene_name': 'gene_name'}, inplace=True)
Downstream_df = Downstream_df.dropna(subset=['gene_name'])
Downstream_df = Downstream_df.groupby('gene_name', as_index=False).mean()
Downstream_df = Downstream_df.sort_values(by=['gene_name']).reset_index(drop=True)
# Find indices where 'row_name' is 'unknown'
Downstream_indices_to_drop = Downstream_df[Downstream_df['gene_name'] == 'unknown'].index
Downstream_df.drop(Downstream_indices_to_drop, inplace=True)

display(Upstream_df)
display(Distal_Promoter_df)
display(Proximal_Promoter_df)
display(Core_Promoter_df)
display(Downstream_df)

,gene_name,Closest_TSS,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,1/2-SBSRNA4,110354972.0,0.887179,0.928626,0.850812,0.917693,0.935774,0.902070,0.918587,0.902734,...,0.874860,0.941140,0.926679,0.946450,0.956023,0.950613,0.957882,0.954680,0.952909,0.937222
1,40969,220960038.0,0.784740,0.775709,0.834890,0.725222,0.827598,0.861398,0.882685,0.869644,...,0.842396,0.904060,0.861046,0.889344,0.539646,0.899930,0.874607,0.764133,0.870613,0.892056
2,40970,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,40971,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,40972,217236749.0,0.355544,0.282240,0.428226,0.290370,0.302508,0.338336,0.308058,0.248291,...,0.181502,0.164838,0.527341,0.448971,0.126454,0.358979,0.439176,0.321057,0.440319,0.514194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24390,mir-220,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24391,mir-29b-2,207996047.0,0.629452,0.578211,0.578683,0.205178,0.576863,0.342100,0.072495,0.357384,...,0.296760,0.588940,0.460734,0.492471,0.550044,0.116830,0.310079,0.361955,0.442842,0.439622
24392,mir-34,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24393,pp9284,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,gene_name,Closest_TSS,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,1/2-SBSRNA4,1.103550e+08,0.073423,0.076605,0.062951,0.058480,0.054831,0.087490,0.059468,0.058524,...,0.040990,0.031435,0.033216,0.042365,0.026143,0.032333,0.034950,0.039405,0.035195,0.031447
1,40969,1.804832e+08,0.402443,0.416247,0.525174,0.495391,0.466687,0.502599,0.517905,0.505021,...,0.465898,0.443598,0.466205,0.439558,0.307342,0.437940,0.360470,0.273794,0.361577,0.360454
2,40970,2.209217e+08,0.073799,0.102480,0.145640,0.218704,0.098510,0.108552,0.111567,0.076365,...,0.047488,0.044825,0.072895,0.073810,0.028841,0.067271,0.059747,0.046542,0.032482,0.058266
3,40971,1.263664e+08,0.388193,0.420424,0.474874,0.432251,0.327370,0.531483,0.394259,0.425125,...,0.514042,0.524002,0.336141,0.552429,0.587300,0.436751,0.470940,0.466759,0.499208,0.448943
4,40972,2.172367e+08,0.255902,0.441512,0.480186,0.447521,0.211151,0.324581,0.370558,0.111229,...,0.113468,0.133592,0.123522,0.406632,0.065165,0.157489,0.192025,0.218505,0.228116,0.283230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24390,mir-220,1.226961e+08,0.860280,0.852637,0.815211,0.866820,0.850400,0.877078,0.886256,0.854447,...,0.783189,0.882374,0.894515,0.878397,0.811522,0.850439,0.917601,0.922466,0.912416,0.912359
24391,mir-29b-2,2.079960e+08,0.193990,0.173256,0.279375,0.314980,0.273213,0.220478,0.206419,0.127361,...,0.549745,0.512224,0.563101,0.304672,0.220729,0.271446,0.199030,0.207845,0.198714,0.248421
24392,mir-34,9.242450e+06,0.346537,0.408150,0.415410,0.429472,0.322381,0.418455,0.293504,0.397016,...,0.544777,0.373107,0.376457,0.392326,0.414736,0.368284,0.368819,0.428284,0.463307,0.408482
24393,pp9284,4.283901e+07,0.443178,0.461584,0.737883,0.807522,0.773087,0.819197,0.716862,0.826073,...,0.765095,0.667561,0.698660,0.817212,0.770317,0.656672,0.517563,0.374266,0.377669,0.551472


,gene_name,Closest_TSS,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,1/2-SBSRNA4,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,40969,1.745803e+08,0.363332,0.535910,0.395403,0.407101,0.435459,0.359165,0.467889,0.231081,...,0.487396,0.370551,0.374183,0.315062,0.261218,0.385854,0.263698,0.125958,0.184370,0.324962
2,40970,1.298745e+08,0.054113,0.048642,0.083515,0.097354,0.053277,0.065180,0.043564,0.040600,...,0.028111,0.021112,0.032823,0.029573,0.032280,0.026998,0.024570,0.027176,0.025910,0.023748
3,40971,1.263664e+08,0.072458,0.065217,0.069581,0.057242,0.055525,0.080413,0.056729,0.053015,...,0.040253,0.029399,0.035392,0.034551,0.024284,0.037558,0.033651,0.034121,0.037867,0.031272
4,40972,2.172367e+08,0.103666,0.113179,0.506935,0.395655,0.082955,0.087271,0.288151,0.083158,...,0.072408,0.058960,0.046124,0.059989,0.034613,0.044347,0.041224,0.044881,0.048920,0.038511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24390,mir-220,1.226961e+08,0.850800,0.847936,0.797880,0.806151,0.761043,0.861510,0.862731,0.802514,...,0.773185,0.873362,0.861938,0.847597,0.535558,0.649543,0.913011,0.907425,0.911604,0.918889
24391,mir-29b-2,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24392,mir-34,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24393,pp9284,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,gene_name,Closest_TSS,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,1/2-SBSRNA4,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,40969,1.931322e+08,0.142585,0.258723,0.188240,0.156086,0.179122,0.268502,0.206790,0.111715,...,0.216817,0.088498,0.146275,0.099898,0.056719,0.103717,0.091091,0.033205,0.116802,0.067281
2,40970,4.388543e+07,0.054452,0.040445,0.030609,0.040729,0.025342,0.046895,0.024501,0.023070,...,0.040649,0.023384,0.019897,0.033009,0.020635,0.020039,0.019098,0.020682,0.021683,0.019699
3,40971,1.263664e+08,0.043689,0.044095,0.019061,0.019090,0.010633,0.038725,0.017218,0.018251,...,0.028957,0.022008,0.017920,0.020176,0.016431,0.017764,0.017366,0.020064,0.022027,0.019995
4,40972,2.172367e+08,0.094279,0.083475,0.475405,0.406994,0.105316,0.100110,0.348689,0.100845,...,0.052682,0.069722,0.039530,0.160281,0.036618,0.144175,0.047691,0.171201,0.268178,0.055187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24390,mir-220,1.226961e+08,0.881067,0.900717,0.799296,0.824275,0.749393,0.832657,0.862455,0.830873,...,0.903724,0.923626,0.914637,0.946778,0.733890,0.663236,0.946871,0.942974,0.943532,0.944827
24391,mir-29b-2,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24392,mir-34,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24393,pp9284,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,gene_name,Closest_TSS,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,1/2-SBSRNA4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,40969,202151617.0,0.165330,0.299532,0.198507,0.223708,0.136140,0.167544,0.143258,0.268029,...,0.116480,0.096383,0.253135,0.185416,0.056247,0.137018,0.185846,0.138916,0.173849,0.182079
2,40970,220921675.0,0.077691,0.082634,0.202797,0.266912,0.103004,0.096773,0.084210,0.062749,...,0.082944,0.065040,0.089758,0.066092,0.038976,0.061406,0.051703,0.051595,0.040429,0.053606
3,40971,126366439.0,0.066413,0.073952,0.083199,0.064017,0.057097,0.073739,0.057288,0.056963,...,0.045191,0.036186,0.038526,0.048280,0.029269,0.028657,0.035230,0.036669,0.032594,0.036779
4,40972,217236749.0,0.201711,0.106647,0.511136,0.451290,0.278742,0.248217,0.578444,0.277375,...,0.283108,0.273946,0.168307,0.203947,0.152229,0.129814,0.185958,0.246684,0.274288,0.173422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24390,mir-220,122696054.0,0.872439,0.872183,0.880797,0.869915,0.852263,0.911205,0.914353,0.886010,...,0.873319,0.910217,0.938888,0.902688,0.937369,0.755657,0.943062,0.967342,0.940335,0.954102
24391,mir-29b-2,207996047.0,0.298442,0.478060,0.727027,0.722579,0.506406,0.390337,0.254398,0.452797,...,0.843802,0.808627,0.843512,0.721485,0.717751,0.446622,0.407509,0.322583,0.302118,0.443711
24392,mir-34,9242450.0,0.211965,0.166876,0.245605,0.270616,0.215078,0.238302,0.175355,0.240629,...,0.204764,0.214735,0.309371,0.250158,0.273429,0.185918,0.250326,0.240734,0.292655,0.283761
24393,pp9284,42839014.0,0.854644,0.866560,0.783603,0.831255,0.822010,0.852905,0.858966,0.841344,...,0.850230,0.836917,0.872262,0.888394,0.873740,0.898982,0.872496,0.890693,0.879333,0.889217


In [55]:
Upstream_df.head()

,gene_name,Closest_TSS,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,1/2-SBSRNA4,110354972.0,0.887179,0.928626,0.850812,0.917693,0.935774,0.902070,0.918587,0.902734,...,0.874860,0.941140,0.926679,0.946450,0.956023,0.950613,0.957882,0.954680,0.952909,0.937222
1,40969,220960038.0,0.784740,0.775709,0.834890,0.725222,0.827598,0.861398,0.882685,0.869644,...,0.842396,0.904060,0.861046,0.889344,0.539646,0.899930,0.874607,0.764133,0.870613,0.892056
2,40970,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,40971,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,40972,217236749.0,0.355544,0.282240,0.428226,0.290370,0.302508,0.338336,0.308058,0.248291,...,0.181502,0.164838,0.527341,0.448971,0.126454,0.358979,0.439176,0.321057,0.440319,0.514194


In [56]:
mutation = mutation[mutation['gene_name'].isin(ensembl_data_unique_gene['Gene name'])]
gene_expression = gene_expression[gene_expression['gene_name'].isin(ensembl_data_unique_gene['Gene name'])]
Upstream_df = Upstream_df[Upstream_df['gene_name'].isin(ensembl_data_unique_gene['Gene name'])]
protein = protein[protein['gene_name'].isin(ensembl_data_unique_gene['Gene name'])]

In [57]:
import pandas as pd

# Convert the gene name columns from each DataFrame to sets
mutation_genes = set(mutation['gene_name'])
print(f"Number of mutation genes: {len(mutation_genes)}")
gene_expression_genes = set(gene_expression['gene_name'])
print(f"Number of gene expression genes: {len(gene_expression_genes)}")
methylation_genes = set(Upstream_df['gene_name'])
print(f"Number of methylation genes: {len(methylation_genes)}")
protein_genes = set(protein['gene_name'])
print(f"Number of protein genes: {len(protein_genes)}")
# Find the intersection of the three sets
common_genes = mutation_genes | gene_expression_genes | methylation_genes | protein_genes

# Convert the intersection back to a list, if needed
common_genes_list = list(common_genes)

# Print the number of common genes
print(f"Number of common genes: {len(common_genes)}")

Number of mutation genes: 30275
Number of gene expression genes: 33248
Number of methylation genes: 18520
Number of protein genes: 87
Number of common genes: 34366


In [59]:
#count the Transcript type
ensembl_data_type = pd.read_csv("./UCSC-raw/Meta-Data/mart_export.txt")
ensembl_data_type= ensembl_data_type.rename(columns={'Gene name': 'gene_name'})
ensembl_data_type = ensembl_data_type.drop_duplicates(subset='gene_name', keep='first')
ensembl_data_type
# Now, merge the two dataframes on the 'gene_name' column
Upstream_df_transcript = pd.merge(Upstream_df, ensembl_data_type, on='gene_name', how='left')
protein_coding_count = (Upstream_df_transcript['Transcript type'] == 'protein_coding').sum()
print(f'Number of rows with "protein_coding": {protein_coding_count}')

protein_transcript = pd.merge(protein, ensembl_data_type, on='gene_name', how='left')
protein_coding_count = (protein_transcript['Transcript type'] == 'protein_coding').sum()
print(f'Number of rows with "protein_coding": {protein_coding_count}')

mutation_transcript = pd.merge(mutation, ensembl_data_type, on='gene_name', how='left')
protein_coding_count = (mutation_transcript['Transcript type'] == 'protein_coding').sum()
print(f'Number of rows with "protein_coding": {protein_coding_count}')

gene_expression_transcript = pd.merge(gene_expression, ensembl_data_type, on='gene_name', how='left')
protein_coding_count = (gene_expression_transcript['Transcript type'] == 'protein_coding').sum()
print(f'Number of rows with "protein_coding": {protein_coding_count}')

Number of rows with "protein_coding": 13651
Number of rows with "protein_coding": 67
Number of rows with "protein_coding": 14651
Number of rows with "protein_coding": 14828


#### 4.1.1 Make the gene expression, mutation, copy number and proteomics data inputted

In [60]:
common_genes_df = pd.DataFrame(common_genes, columns=['gene_name'])
common_genes_df = common_genes_df.sort_values(by=['gene_name']).reset_index(drop=True)
common_genes_df

,gene_name
0,5S_rRNA
1,5_8S_rRNA
2,7SK
3,A1BG
4,A1BG-AS1
...,...
34361,ZYXP1
34362,ZZEF1
34363,ZZZ3
34364,hsa-mir-1253


In [61]:
merged_data_transcript_Upstream_df = pd.merge(common_genes_df, ensembl_data_type, on='gene_name', how='left')
protein_coding_count = (merged_data_transcript_Upstream_df['Transcript type'] == 'protein_coding').sum()
print(f'Number of rows with "protein_coding": {protein_coding_count}')

Number of rows with "protein_coding": 14866


In [62]:
unique_values = merged_data_transcript_Upstream_df['Transcript type'].value_counts()
unique_values

protein_coding                        14866
processed_pseudogene                   5312
lncRNA                                 2407
miRNA                                  1817
snRNA                                  1813
protein_coding_CDS_not_defined         1342
unprocessed_pseudogene                 1286
misc_RNA                               1035
nonsense_mediated_decay                1031
retained_intron                         895
transcribed_unprocessed_pseudogene      525
rRNA_pseudogene                         490
snoRNA                                  388
transcribed_processed_pseudogene        264
IG_V_pseudogene                         182
IG_V_gene                               138
TR_V_gene                               106
TR_J_gene                                79
transcribed_unitary_pseudogene           69
unitary_pseudogene                       59
IG_D_gene                                37
TR_V_pseudogene                          33
protein_coding_LoF              

In [63]:
gene_expression_inputted = pd.merge(common_genes_df ,gene_expression,on='gene_name',how='outer').fillna(0)
protein_inputted = pd.merge(common_genes_df, protein, on='gene_name', how='outer').fillna(0)

Upstream_df_inputted = pd.merge(common_genes_df, Upstream_df,on='gene_name',how='outer').fillna(0)
Distal_Promoter_df_inputted = pd.merge(common_genes_df, Distal_Promoter_df,on='gene_name',how='outer').fillna(0)
Proximal_Promoter_df_inputted = pd.merge(common_genes_df, Proximal_Promoter_df,on='gene_name',how='outer').fillna(0)
Core_Promoter_df_inputted = pd.merge(common_genes_df, Core_Promoter_df,on='gene_name',how='outer').fillna(0)
Downstream_df_inputted = pd.merge(common_genes_df, Downstream_df,on='gene_name',how='outer').fillna(0)

mutation_inputted = pd.merge(common_genes_df, mutation, on='gene_name', how='outer').fillna(-1)

display(gene_expression_inputted)

,gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,5S_rRNA,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.965800,-9.9658,-9.965800,...,-7.912300,-9.168762,-9.9658,-9.9658,-9.9658,-9.965800,-9.9658,-9.9658,-9.9658,-9.9658
1,5_8S_rRNA,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.965800,-9.9658,-9.965800,...,-9.965800,-9.965800,-9.9658,-9.9658,-9.9658,-9.965800,-9.9658,-9.9658,-9.9658,-9.9658
2,7SK,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-8.556583,-9.9658,-7.915867,...,-6.844283,-8.977333,-9.9658,-9.9658,-9.9658,-7.988867,-9.9658,-9.9658,-9.9658,-9.9658
3,A1BG,4.1692,1.0642,4.4047,3.3883,3.4907,6.5571,3.627700,2.7205,4.557700,...,1.575800,-0.150400,1.8957,5.7528,5.0867,1.700900,2.8321,1.6374,1.2023,2.1444
4,A1BG-AS1,2.0325,-0.7346,2.3479,1.0711,1.0363,2.4306,-0.575600,0.4552,2.682800,...,-0.413100,-0.859900,-0.0574,2.6984,2.5137,-0.042500,0.2522,-1.3183,-0.9686,-0.6643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34361,ZYXP1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.965800,-9.9658,-9.965800,...,-9.965800,-9.965800,-9.9658,-9.9658,-9.9658,-9.965800,-9.9658,-9.9658,-9.9658,-9.9658
34362,ZZEF1,3.1556,3.1179,3.0181,3.0074,2.3926,1.1641,1.157700,1.9490,3.136000,...,3.403400,3.323500,2.2513,1.5316,1.4600,3.835000,2.2573,2.3620,2.2051,2.6067
34363,ZZZ3,3.3549,3.6195,3.3619,3.3105,2.6624,3.4635,3.744300,2.7972,3.497100,...,3.611300,2.815800,3.4583,3.2870,2.9243,3.521200,3.0876,3.3192,2.7805,3.5212
34364,hsa-mir-1253,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.965800,-9.9658,-9.965800,...,-9.965800,-9.965800,-9.9658,-9.9658,-9.9658,-9.965800,-9.9658,-9.9658,-9.9658,-9.9658


#### 4.1.2 Intersecting genes with various databases

In [66]:
import pandas as pd
# Add the gene names from databases like [KEGG / BioGRID] to intersect with the common genes
# KEGG
kegg_pathway_df = pd.read_csv('./Regulatory-network-data/KEGG/full_kegg_pathway_list.csv')
kegg_pathway_df = kegg_pathway_df[['source', 'target', 'pathway_name']]
kegg_df = kegg_pathway_df[kegg_pathway_df['pathway_name'].str.contains('signaling pathway|signaling pathways', case=False)]
print(kegg_df['pathway_name'].value_counts())
kegg_df = kegg_df.rename(columns={'source': 'src', 'target': 'dest'})
src_list = list(kegg_df['src'])
dest_list = list(kegg_df['dest'])
path_list = list(kegg_df['pathway_name'])
# ADJUST ALL GENES TO UPPERCASE
up_src_list = []
for src in src_list:
    up_src = src.upper()
    up_src_list.append(up_src)
up_dest_list = []
for dest in dest_list:
    up_dest = dest.upper()
    up_dest_list.append(up_dest)
up_kegg_conn_dict = {'src': up_src_list, 'dest': up_dest_list}
up_kegg_df = pd.DataFrame(up_kegg_conn_dict)
up_kegg_df = up_kegg_df.drop_duplicates()
up_kegg_df.to_csv('./Regulatory-network-data/KEGG/up_kegg.csv', index=False, header=True)
kegg_gene_list = list(set(list(up_kegg_df['src']) + list(up_kegg_df['dest'])))
print('----- NUMBER OF GENES IN KEGG: ' + str(len(kegg_gene_list)) + ' -----')
print(up_kegg_df.shape)

up_kegg_path_conn_dict = {'src': up_src_list, 'dest': up_dest_list, 'path': path_list}
up_kegg_path_df = pd.DataFrame(up_kegg_path_conn_dict)
up_kegg_path_df = up_kegg_path_df.drop_duplicates()
up_kegg_path_df.to_csv('./Regulatory-network-data/KEGG/up_kegg_path.csv', index=False, header=True)
kegg_path_gene_list = list(set(list(up_kegg_path_df['src']) + list(up_kegg_path_df['dest'])))
print('----- NUMBER OF GENES IN KEGG PATH: ' + str(len(kegg_path_gene_list)) + ' -----')
print(up_kegg_path_df.shape)

PI3K-Akt signaling pathway                                  3992
JAK-STAT signaling pathway                                  3280
Chemokine signaling pathway                                 2766
MAPK signaling pathway                                      2002
cAMP signaling pathway                                      1680
Ras signaling pathway                                       1640
Rap1 signaling pathway                                      1304
Calcium signaling pathway                                   1260
Apelin signaling pathway                                    1011
Wnt signaling pathway                                        935
mTOR signaling pathway                                       836
Hippo signaling pathway                                      830
Insulin signaling pathway                                    721
Glucagon signaling pathway                                   707
Relaxin signaling pathway                                    684
Phospholipase D signaling

In [67]:
# BioGRID
biogrid_df = pd.read_table('./Regulatory-network-data/BioGrid/BIOGRID-ALL-3.5.174.mitab.Symbol.txt', delimiter = '\t')
eh_list = list(biogrid_df['e_h'])
et_list = list(biogrid_df['e_t'])
# ADJUST ALL GENES TO UPPERCASE
up_eh_list = []
for eh in eh_list:
    up_eh = eh.upper()
    up_eh_list.append(up_eh)
up_et_list = []
for et in et_list:
    up_et = et.upper()
    up_et_list.append(up_et)
up_biogrid_conn_dict = {'src': up_eh_list, 'dest': up_et_list}
up_biogrid_df = pd.DataFrame(up_biogrid_conn_dict)
print(up_biogrid_df)
print(up_biogrid_df.shape)
up_biogrid_df.to_csv('./Regulatory-network-data/BioGrid/up_biogrid.csv', index = False, header = True)
up_biogrid_gene_list = list(set(list(up_biogrid_df['src']) + list(up_biogrid_df['dest'])))
print('----- NUMBER OF GENES IN BioGRID: ' + str(len(up_biogrid_gene_list)) + ' -----')

           src    dest
0       MAP2K4    FLNC
1         MYPN   ACTN2
2        ACVR1    FNTA
3        GATA2     PML
4         RPA2   STAT3
...        ...     ...
472638   USP18  SAMHD1
472639   USP18    SKP2
472640  SAMHD1   USP18
472641  SAMHD1   CCNA2
472642  SAMHD1    CDK1

[472643 rows x 2 columns]
(472643, 2)
----- NUMBER OF GENES IN BioGRID: 19349 -----


In [68]:
# STRING
string_df = pd.read_csv('./Regulatory-network-data/STRING/9606.protein.links.detailed.v11.0_sym.csv', low_memory=False)
src_list = list(string_df['Source'])
tar_list = list(string_df['Target'])
# ADJUST ALL GENES TO UPPERCASE
up_src_list = []
for src in src_list:
    up_src = src.upper()
    up_src_list.append(up_src)
up_tar_list = []
for tar in tar_list:
    up_tar = tar.upper()
    up_tar_list.append(up_tar)
up_string_conn_dict = {'src': up_src_list, 'dest': up_tar_list}
up_string_df = pd.DataFrame(up_string_conn_dict)
print(up_string_df)
up_string_df.to_csv('./Regulatory-network-data/STRING/up_string.csv', index = False, header = True)
up_string_gene_list = list(set(list(up_string_df['src']) + list(up_string_df['dest'])))
print('----- NUMBER OF GENES IN STRING: ' + str(len(up_string_gene_list)) + ' -----')

          src    dest
0        ARF5  SPTBN2
1        ARF5  KIF13B
2        ARF5   AP1B1
3        ARF5  KIF21A
4        ARF5   TMED7
...       ...     ...
841063  OR6Q1   REEP1
841064  OR6Q1   REEP4
841065  OR6Q1    GNB1
841066  OR6Q1    RTP3
841067  OR6Q1   REEP2

[841068 rows x 2 columns]
----- NUMBER OF GENES IN STRING: 17179 -----


In [69]:
# intersect the [common genes] with the genes in the different databases [KEGG / BioGRID / STRING]
selected_database = 'KEGG'
# selected_database = 'BioGRID'
# selected_database = 'STRING'
if selected_database == 'KEGG':
    edge_common_genes = list(set(common_genes) & set(kegg_gene_list))
    print('----- NUMBER OF INTERSECTED GENES IN KEGG: ' + str(len(edge_common_genes)) + ' -----')
elif selected_database == 'BioGRID':
    edge_common_genes = list(set(common_genes) & set(up_biogrid_gene_list))
    print('----- NUMBER OF INTERSECTED GENES IN BioGRID: ' + str(len(edge_common_genes)) + ' -----')
elif selected_database == 'STRING':
    edge_common_genes = list(set(common_genes) & set(up_string_gene_list))
    print('----- NUMBER OF INTERSECTED GENES IN STRING: ' + str(len(edge_common_genes)) + ' -----')

# filter the genes in the different databases [KEGG / BioGRID / STRING] with the [common genes]
if selected_database == 'KEGG':
    filtered_up_kegg_df = up_kegg_df[up_kegg_df['src'].isin(edge_common_genes) & up_kegg_df['dest'].isin(edge_common_genes)]
    filtered_up_kegg_df = filtered_up_kegg_df.drop_duplicates()
    filtered_up_kegg_df = filtered_up_kegg_df.sort_values(by=['src', 'dest']).reset_index(drop=True)
    print('----- NEW KEGG EDGE CONNECTIONS: ' + str(len(filtered_up_kegg_df)) + ' -----')
    filtered_up_kegg_path_df = up_kegg_path_df[up_kegg_path_df['src'].isin(edge_common_genes) & up_kegg_path_df['dest'].isin(edge_common_genes)]
    filtered_up_kegg_path_df = filtered_up_kegg_path_df.drop_duplicates()
    filtered_up_kegg_path_df = filtered_up_kegg_path_df.sort_values(by=['src', 'dest']).reset_index(drop=True)
    print('----- NEW KEGG PATHWAY CONNECTIONS: ' + str(len(filtered_up_kegg_path_df)) + ' -----')
elif selected_database == 'BioGRID':
    filtered_up_biogrid_df = up_biogrid_df[up_biogrid_df['src'].isin(edge_common_genes) & up_biogrid_df['dest'].isin(edge_common_genes)]
    filtered_up_biogrid_df = filtered_up_biogrid_df.drop_duplicates()
    filtered_up_biogrid_df = filtered_up_biogrid_df.sort_values(by=['src', 'dest']).reset_index(drop=True)
    print('----- NEW BioGRID EDGE CONNECTIONS: ' + str(len(filtered_up_biogrid_df)) + ' -----')
elif selected_database == 'STRING':
    filtered_up_string_df = up_string_df[up_string_df['src'].isin(edge_common_genes) & up_string_df['dest'].isin(edge_common_genes)]
    filtered_up_string_df = filtered_up_string_df.drop_duplicates()
    filtered_up_string_df = filtered_up_string_df.sort_values(by=['src', 'dest']).reset_index(drop=True)
    print('----- NEW STRING EDGE CONNECTIONS: ' + str(len(filtered_up_string_df)) + ' -----')

----- NUMBER OF INTERSECTED GENES IN KEGG: 2117 -----
----- NEW KEGG EDGE CONNECTIONS: 19694 -----
----- NEW KEGG PATHWAY CONNECTIONS: 28671 -----


In [70]:
if selected_database == 'KEGG':
    display(filtered_up_kegg_df)
    display(filtered_up_kegg_path_df)
elif selected_database == 'BioGRID':
    display(filtered_up_biogrid_df)
elif selected_database == 'STRING':
    display(filtered_up_string_df)

,src,dest
0,ACTB,MYL6
1,ACTB,MYL6B
2,ACTB,MYL9
3,ACTB,STK3
4,ACTG1,MYL6
...,...,...
19689,ZNRF3,FZD5
19690,ZNRF3,FZD6
19691,ZNRF3,FZD7
19692,ZNRF3,FZD8


,src,dest,path
0,ACTB,MYL6,Oxytocin signaling pathway
1,ACTB,MYL6B,Oxytocin signaling pathway
2,ACTB,MYL9,Oxytocin signaling pathway
3,ACTB,STK3,Hippo signaling pathway
4,ACTG1,MYL6,Oxytocin signaling pathway
...,...,...,...
28666,ZNRF3,FZD5,Wnt signaling pathway
28667,ZNRF3,FZD6,Wnt signaling pathway
28668,ZNRF3,FZD7,Wnt signaling pathway
28669,ZNRF3,FZD8,Wnt signaling pathway


#### 4.1.3 Filtering the gene names across the gene expression, cnv, proteomics and methylation

In [71]:
# select common genes in mutation data
mutation_filtered = mutation_inputted.loc[mutation_inputted['gene_name'].isin(edge_common_genes)]
mutation_filtered = mutation_filtered.sort_values(by=['gene_name']).reset_index(drop=True)
# select common genes in gene expression data
gene_expression_filtered = gene_expression_inputted.loc[gene_expression_inputted['gene_name'].isin(edge_common_genes)]
gene_expression_filtered = gene_expression_filtered.sort_values(by=['gene_name']).reset_index(drop=True)
# select common genes in protein data
protein_filtered = protein_inputted.loc[protein_inputted['gene_name'].isin(edge_common_genes)]
protein_filtered = protein_filtered.sort_values(by=['gene_name']).reset_index(drop=True)
# select common genes in upstream data
Upstream_df_filtered = Upstream_df_inputted.loc[Upstream_df_inputted['gene_name'].isin(edge_common_genes)]
Upstream_df_filtered = Upstream_df_filtered.sort_values(by=['gene_name']).reset_index(drop=True)
# select common genes in distal promoter data
Distal_Promoter_df_filtered = Distal_Promoter_df_inputted.loc[Distal_Promoter_df_inputted['gene_name'].isin(edge_common_genes)]
Distal_Promoter_df_filtered = Distal_Promoter_df_filtered.sort_values(by=['gene_name']).reset_index(drop=True)
# select common genes in proximal promoter data
Proximal_Promoter_df_filtered = Proximal_Promoter_df_inputted.loc[Proximal_Promoter_df_inputted['gene_name'].isin(edge_common_genes)]
Proximal_Promoter_df_filtered = Proximal_Promoter_df_filtered.sort_values(by=['gene_name']).reset_index(drop=True)
# select common genes in core promoter data
Core_Promoter_df_filtered = Core_Promoter_df_inputted.loc[Core_Promoter_df_inputted['gene_name'].isin(edge_common_genes)]
Core_Promoter_df_filtered = Core_Promoter_df_filtered.sort_values(by=['gene_name']).reset_index(drop=True)
# select common genes in downstream data
Downstream_df_filtered = Downstream_df_inputted.loc[Downstream_df_inputted['gene_name'].isin(edge_common_genes)]
Downstream_df_filtered = Downstream_df_filtered.sort_values(by=['gene_name']).reset_index(drop=True)

In [72]:
mutation_filtered

,gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,ABL1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABL2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACAA1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACACA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACACB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,ZFYVE16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2113,ZFYVE9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2114,ZMAT3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2115,ZNF274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
#count the Transcript type
common_genes_transcript = pd.merge(mutation_filtered, ensembl_data_type, on='gene_name', how='left')
protein_coding_count = (common_genes_transcript['Transcript type'] == 'protein_coding').sum()
print(f'Number of rows with "protein_coding": {protein_coding_count}')

Number of rows with "protein_coding": 1765


In [74]:
unique_values = common_genes_transcript['Transcript type'].value_counts()
unique_values

protein_coding                    1765
protein_coding_CDS_not_defined     135
retained_intron                    117
nonsense_mediated_decay             97
non_stop_decay                       2
lncRNA                               1
Name: Transcript type, dtype: int64

## 5.Gene name/patient samples/ pheotype file lists

### 5.1 gene name and patient samples lists

In [75]:
gene_list = gene_expression_filtered['gene_name']
gene_list

0          ABL1
1          ABL2
2         ACAA1
3         ACACA
4         ACACB
         ...   
2112    ZFYVE16
2113     ZFYVE9
2114      ZMAT3
2115     ZNF274
2116      ZNRF3
Name: gene_name, Length: 2117, dtype: object

In [76]:
protein_list = protein_filtered['gene_name'].tolist()
print(len(protein_list))
protein_list

2117


['ABL1',
 'ABL2',
 'ACAA1',
 'ACACA',
 'ACACB',
 'ACADL',
 'ACADM',
 'ACOX1',
 'ACOX2',
 'ACOX3',
 'ACSBG1',
 'ACSBG2',
 'ACSL1',
 'ACSL3',
 'ACSL4',
 'ACSL5',
 'ACSL6',
 'ACTA2',
 'ACTB',
 'ACTG1',
 'ACVR1',
 'ACVR1B',
 'ACVR1C',
 'ACVR2A',
 'ACVR2B',
 'ADAM17',
 'ADCY1',
 'ADCY10',
 'ADCY2',
 'ADCY3',
 'ADCY4',
 'ADCY5',
 'ADCY6',
 'ADCY7',
 'ADCY8',
 'ADCY9',
 'ADCYAP1',
 'ADCYAP1R1',
 'ADGRB1',
 'ADIPOQ',
 'ADIPOR1',
 'ADIPOR2',
 'ADORA1',
 'ADORA2A',
 'ADORA2B',
 'ADORA3',
 'ADRA1A',
 'ADRA1B',
 'ADRA1D',
 'ADRA2A',
 'ADRA2B',
 'ADRA2C',
 'ADRB1',
 'ADRB2',
 'ADRB3',
 'AFP',
 'AGAP2',
 'AGER',
 'AGRP',
 'AGT',
 'AGTR1',
 'AIFM2',
 'AIM2',
 'AJUBA',
 'AKT1',
 'AKT1S1',
 'AKT2',
 'AKT3',
 'ALDOA',
 'ALDOB',
 'ALDOC',
 'AMH',
 'AMHR2',
 'AMOT',
 'ANAPC5',
 'ANGPT1',
 'ANGPT2',
 'ANGPT4',
 'ANGPTL4',
 'AOX1',
 'APAF1',
 'APBB1IP',
 'APC',
 'APC2',
 'APH1A',
 'APH1B',
 'APLN',
 'APLNR',
 'APOA1',
 'APOA2',
 'APOA5',
 'APOC3',
 'AQP7',
 'ARAF',
 'ARAP3',
 'AREG',
 'ARF1',
 'ARF6',
 'ARH

In [77]:
intersection = list(set(gene_list) & set(protein_list))
len(intersection)

2117

In [78]:
patient_sample_list = pd.DataFrame(common_tcga_columns_list,columns=['sample'])
patient_sample_list

,sample
0,TCGA-05-4384-01
1,TCGA-05-4396-01
2,TCGA-05-4405-01
3,TCGA-05-4410-01
4,TCGA-05-4417-01
...,...
3587,TCGA-ZA-A8F6-01
3588,TCGA-ZG-A8QW-01
3589,TCGA-ZG-A8QX-01
3590,TCGA-ZG-A8QY-01


### 5.2 phenotype lists

In [79]:
immune_subtype_filtered

,sample,Subtype_Immune_Model_Based
0,TCGA-05-4384-01,Inflammatory (Immune C3)
1,TCGA-05-4396-01,Inflammatory (Immune C3)
2,TCGA-05-4405-01,Inflammatory (Immune C3)
3,TCGA-05-4410-01,Inflammatory (Immune C3)
4,TCGA-05-4417-01,Inflammatory (Immune C3)
...,...,...
3587,TCGA-ZA-A8F6-01,Inflammatory (Immune C3)
3588,TCGA-ZG-A8QW-01,Inflammatory (Immune C3)
3589,TCGA-ZG-A8QX-01,Inflammatory (Immune C3)
3590,TCGA-ZG-A8QY-01,Inflammatory (Immune C3)


In [80]:
survival_filtered

,sample,_PATIENT,cancer type abbreviation,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,residual_tumor,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction
0,TCGA-05-4384-01,TCGA-05-4384,LUAD,66.0,MALE,NaN,Stage IIIA,NaN,Lung Adenocarcinoma,NaN,...,NaN,0.0,426.0,0.0,426.0,NaN,NaN,1.0,183.0,NaN
1,TCGA-05-4396-01,TCGA-05-4396,LUAD,76.0,MALE,NaN,Stage IIIB,NaN,Lung Adenocarcinoma,NaN,...,NaN,1.0,303.0,NaN,303.0,NaN,NaN,0.0,303.0,NaN
2,TCGA-05-4405-01,TCGA-05-4405,LUAD,74.0,FEMALE,NaN,Stage IB,NaN,Lung Adenocarcinoma,NaN,...,NaN,0.0,610.0,0.0,610.0,NaN,NaN,0.0,610.0,NaN
3,TCGA-05-4410-01,TCGA-05-4410,LUAD,62.0,MALE,NaN,Stage IB,NaN,Lung Adenocarcinoma,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN
4,TCGA-05-4417-01,TCGA-05-4417,LUAD,51.0,FEMALE,NaN,Stage IB,NaN,Lung Adenocarcinoma,NaN,...,NaN,0.0,455.0,0.0,455.0,NaN,NaN,0.0,455.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587,TCGA-ZA-A8F6-01,TCGA-ZA-A8F6,STAD,71.0,MALE,WHITE,Stage IB,NaN,"Stomach, Intestinal Adenocarcinoma, Not Otherw...",G2,...,NaN,0.0,525.0,0.0,525.0,0.0,525.0,0.0,525.0,NaN
3588,TCGA-ZG-A8QW-01,TCGA-ZG-A8QW,PRAD,72.0,MALE,NaN,NaN,NaN,Prostate Adenocarcinoma Acinar Type,NaN,...,NaN,0.0,94.0,0.0,94.0,NaN,NaN,0.0,94.0,NaN
3589,TCGA-ZG-A8QX-01,TCGA-ZG-A8QX,PRAD,56.0,MALE,NaN,NaN,NaN,Prostate Adenocarcinoma Acinar Type,NaN,...,NaN,0.0,442.0,0.0,442.0,NaN,NaN,0.0,442.0,NaN
3590,TCGA-ZG-A8QY-01,TCGA-ZG-A8QY,PRAD,67.0,MALE,NaN,NaN,NaN,Prostate Adenocarcinoma Acinar Type,NaN,...,NaN,0.0,404.0,0.0,404.0,NaN,NaN,0.0,404.0,NaN


In [81]:
survival_nan_column_proportions = survival_filtered.isna().mean()

# Display the results
print(survival_nan_column_proportions)

sample                                 0.000000
_PATIENT                               0.000000
cancer type abbreviation               0.000000
age_at_initial_pathologic_diagnosis    0.006403
gender                                 0.000000
race                                   0.072940
ajcc_pathologic_tumor_stage            0.263641
clinical_stage                         0.872216
histological_type                      0.016425
histological_grade                     0.570434
initial_pathologic_dx_year             0.007238
menopause_status                       0.823497
birth_days_to                          0.013085
vital_status                           0.000000
tumor_status                           0.055679
last_contact_days_to                   0.240256
death_days_to                          0.748051
cause_of_death                         0.987751
new_tumor_event_type                   0.849109
new_tumor_event_site                   0.896158
new_tumor_event_site_other             0

In [82]:
# Calculate the proportion of NaN values in each column
survival_nan_column_proportions = survival_filtered.isna().mean()

# Identify columns to be dropped (where proportion of NaN values is greater than 1/3)
columns_to_drop = survival_nan_column_proportions[survival_nan_column_proportions > 1/3].index.tolist()

# Drop these columns from the DataFrame
survival_filtered = survival_filtered.drop(columns=columns_to_drop)

# List of columns that were dropped
print("Columns dropped:", columns_to_drop)

Columns dropped: ['clinical_stage', 'histological_grade', 'menopause_status', 'death_days_to', 'cause_of_death', 'new_tumor_event_type', 'new_tumor_event_site', 'new_tumor_event_site_other', 'new_tumor_event_dx_days_to', 'treatment_outcome_first_course', 'margin_status', 'residual_tumor', 'DFI', 'DFI.time', 'Redaction']


In [83]:
cellsub_nan_column_proportions = cellsub_filtered.isna().mean()

# Display the results
print(cellsub_nan_column_proportions)

sample                 0.000000
Subtype_mRNA           0.273664
Subtype_DNAmeth        0.492762
Subtype_protein        0.606904
Subtype_miRNA          0.602728
Subtype_CNA            0.669265
Subtype_Integrative    0.802060
Subtype_other          0.390869
Subtype_Selected       0.000000
dtype: float64


In [84]:
# Calculate the proportion of NaN values in each column
cellsub_nan_column_proportions = cellsub_filtered.isna().mean()

# Identify columns to be dropped (where proportion of NaN values is greater than 1/3)
columns_to_drop = cellsub_nan_column_proportions[cellsub_nan_column_proportions > 1/3].index.tolist()

# Drop these columns from the DataFrame
cellsub_filtered = cellsub_filtered.drop(columns=columns_to_drop)

# List of columns that were dropped
print("Columns dropped:", columns_to_drop)

Columns dropped: ['Subtype_DNAmeth', 'Subtype_protein', 'Subtype_miRNA', 'Subtype_CNA', 'Subtype_Integrative', 'Subtype_other']


In [85]:
cellsub_filtered

,sample,Subtype_mRNA,Subtype_Selected
0,TCGA-05-4384-01,NaN,LUAD.1
1,TCGA-05-4396-01,NaN,LUAD.4
2,TCGA-05-4405-01,NaN,LUAD.3
3,TCGA-05-4410-01,NaN,LUAD.3
4,TCGA-05-4417-01,NaN,LUAD.6
...,...,...,...
3587,TCGA-ZA-A8F6-01,NaN,GI.CIN
3588,TCGA-ZG-A8QW-01,1,PRAD.8-other
3589,TCGA-ZG-A8QX-01,1,PRAD.5-SPOP
3590,TCGA-ZG-A8QY-01,3,PRAD.5-SPOP


In [86]:
import pandas as pd

# extract phenotype names
immune_phenotypes = immune_subtype_filtered.columns[1:].tolist()
survival_phenotypes = survival_filtered.columns[2:].tolist() # _PATIENT infor is not needed (sample id)
dense_phenotypes = dense_filtered.columns[2:].tolist() # sample_type_id infor is not needed (all = 1)
cellsub_phenotypes = cellsub_filtered.columns[1:].tolist()

# creat phenotype name and source
phenotype_list = []
phenotype_list.extend([(p, 'immunesub') for p in immune_phenotypes])
phenotype_list.extend([(p, 'survival') for p in survival_phenotypes])
phenotype_list.extend([(p, 'dense') for p in dense_phenotypes])
phenotype_list.extend([(p, 'cellsub') for p in cellsub_phenotypes])

# list DataFrame
phenotype_lists = pd.DataFrame(phenotype_list, columns=['Phenotype_Name', 'Phenotype_Source'])
phenotype_lists

,Phenotype_Name,Phenotype_Source
0,Subtype_Immune_Model_Based,immunesub
1,cancer type abbreviation,survival
2,age_at_initial_pathologic_diagnosis,survival
3,gender,survival
4,race,survival
5,ajcc_pathologic_tumor_stage,survival
6,histological_type,survival
7,initial_pathologic_dx_year,survival
8,birth_days_to,survival
9,vital_status,survival


## 6.Save processed datasets

### 6.1 Keep the consistency for dataframes on genes and samples

In [87]:
# [gene_list]
# gene-tran
sorted_gene_list = gene_list.sort_values()
sorted_gene = sorted_gene_list.tolist()
sorted_gene_tran = [gene + '-TRAN' for gene in sorted_gene]
sorted_gene_tran_df = pd.DataFrame(sorted_gene_tran, columns=['Gene'])
display(sorted_gene_tran_df)
# gene-meth
sorted_gene_methy = [gene + '-METH' for gene in sorted_gene]
sorted_gene_methy_df = pd.DataFrame(sorted_gene_methy, columns=['Gene'])
display(sorted_gene_methy_df)
# gene-protein
sorted_gene_protein = [gene + '-PROT' for gene in sorted_gene]
sorted_gene_protein_df = pd.DataFrame(sorted_gene_protein, columns=['Gene'])
display(sorted_gene_protein_df)
# all-gene
sorted_gene_all = sorted_gene_tran + sorted_gene_methy + sorted_gene_protein
sorted_all_gene_df = pd.DataFrame(sorted_gene_all, columns=['Gene'])
display(sorted_all_gene_df)

,Gene
0,ABL1-TRAN
1,ABL2-TRAN
2,ACAA1-TRAN
3,ACACA-TRAN
4,ACACB-TRAN
...,...
2112,ZFYVE16-TRAN
2113,ZFYVE9-TRAN
2114,ZMAT3-TRAN
2115,ZNF274-TRAN


,Gene
0,ABL1-METH
1,ABL2-METH
2,ACAA1-METH
3,ACACA-METH
4,ACACB-METH
...,...
2112,ZFYVE16-METH
2113,ZFYVE9-METH
2114,ZMAT3-METH
2115,ZNF274-METH


,Gene
0,ABL1-PROT
1,ABL2-PROT
2,ACAA1-PROT
3,ACACA-PROT
4,ACACB-PROT
...,...
2112,ZFYVE16-PROT
2113,ZFYVE9-PROT
2114,ZMAT3-PROT
2115,ZNF274-PROT


,Gene
0,ABL1-TRAN
1,ABL2-TRAN
2,ACAA1-TRAN
3,ACACA-TRAN
4,ACACB-TRAN
...,...
6346,ZFYVE16-PROT
6347,ZFYVE9-PROT
6348,ZMAT3-PROT
6349,ZNF274-PROT


In [88]:
# [patient-sample-list]
sorted_patient_sample_list = patient_sample_list.sort_values(by='sample')['sample'].tolist()
print(sorted_patient_sample_list)
sorted_patient_sample_df = patient_sample_list.sort_values(by='sample').reset_index(drop=True)
display(sorted_patient_sample_df)

['TCGA-05-4384-01', 'TCGA-05-4396-01', 'TCGA-05-4405-01', 'TCGA-05-4410-01', 'TCGA-05-4417-01', 'TCGA-05-5423-01', 'TCGA-05-5429-01', 'TCGA-05-5715-01', 'TCGA-06-0125-01', 'TCGA-06-0190-01', 'TCGA-06-1804-01', 'TCGA-06-5408-01', 'TCGA-06-5410-01', 'TCGA-06-5411-01', 'TCGA-06-5412-01', 'TCGA-06-5413-01', 'TCGA-06-5414-01', 'TCGA-06-5415-01', 'TCGA-06-5416-01', 'TCGA-06-5417-01', 'TCGA-06-5418-01', 'TCGA-06-5856-01', 'TCGA-06-5858-01', 'TCGA-06-5859-01', 'TCGA-12-5295-01', 'TCGA-18-4721-01', 'TCGA-18-5592-01', 'TCGA-19-4065-01', 'TCGA-19-5960-01', 'TCGA-21-5782-01', 'TCGA-21-5784-01', 'TCGA-21-5786-01', 'TCGA-21-5787-01', 'TCGA-22-5471-01', 'TCGA-22-5472-01', 'TCGA-22-5473-01', 'TCGA-22-5474-01', 'TCGA-22-5477-01', 'TCGA-22-5478-01', 'TCGA-22-5480-01', 'TCGA-22-5482-01', 'TCGA-22-5485-01', 'TCGA-22-5489-01', 'TCGA-22-5491-01', 'TCGA-22-5492-01', 'TCGA-26-1442-01', 'TCGA-26-5132-01', 'TCGA-26-5133-01', 'TCGA-26-5134-01', 'TCGA-28-5208-01', 'TCGA-2A-A8VL-01', 'TCGA-2A-A8VO-01', 'TCGA-2A-A8

,sample
0,TCGA-05-4384-01
1,TCGA-05-4396-01
2,TCGA-05-4405-01
3,TCGA-05-4410-01
4,TCGA-05-4417-01
...,...
3587,TCGA-ZA-A8F6-01
3588,TCGA-ZG-A8QW-01
3589,TCGA-ZG-A8QX-01
3590,TCGA-ZG-A8QY-01


In [89]:
Upstream_df_filtered = Upstream_df_filtered[['gene_name'] + sorted_patient_sample_list]
Distal_Promoter_df_filtered = Distal_Promoter_df_filtered[['gene_name'] + sorted_patient_sample_list]
Proximal_Promoter_df_filtered = Proximal_Promoter_df_filtered[['gene_name'] + sorted_patient_sample_list]
Core_Promoter_df_filtered = Core_Promoter_df_filtered[['gene_name'] + sorted_patient_sample_list]
Downstream_df_filtered = Downstream_df_filtered[['gene_name'] + sorted_patient_sample_list]

display(Upstream_df_filtered)

,gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,ABL1,0.863152,0.872278,0.881857,0.851133,0.869055,0.841541,0.877805,0.860541,0.852925,...,0.819826,0.896648,0.858912,0.844898,0.814890,0.930322,0.936406,0.927422,0.929564,0.909714
1,ABL2,0.878665,0.880215,0.831704,0.854432,0.898190,0.822789,0.842128,0.855198,0.926848,...,0.852741,0.755294,0.932633,0.838991,0.902085,0.896923,0.948045,0.924965,0.892633,0.922470
2,ACAA1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,ACACA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ACACB,0.375306,0.447112,0.463906,0.494918,0.394273,0.327884,0.381452,0.254099,0.130568,...,0.339652,0.521416,0.294107,0.336399,0.597227,0.175013,0.215435,0.347599,0.395659,0.349506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,ZFYVE16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2113,ZFYVE9,0.851893,0.858037,0.831665,0.811753,0.845776,0.863929,0.814258,0.820470,0.785349,...,0.827819,0.891287,0.867332,0.897818,0.800718,0.873442,0.878316,0.893267,0.901400,0.885994
2114,ZMAT3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2115,ZNF274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [90]:
mutation_filtered = mutation_filtered[['gene_name'] + sorted_patient_sample_list].sort_values(by='gene_name').reset_index(drop=True)
mutation_filtered

,gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,ABL1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABL2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACAA1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACACA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACACB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,ZFYVE16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2113,ZFYVE9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2114,ZMAT3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2115,ZNF274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
gene_expression_filtered = gene_expression_filtered[['gene_name'] + sorted_patient_sample_list].sort_values(by='gene_name').reset_index(drop=True)
gene_expression_filtered

,gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,ABL1,4.3667,4.6657,4.4764,4.1400,4.3779,2.9544,3.1876,3.4504,5.1264,...,4.0037,4.7361,4.3421,3.6748,4.8934,4.6201,4.3385,4.3992,4.6456,4.2350
1,ABL2,2.0465,1.1706,2.6325,1.6187,2.2723,0.7748,-0.1993,1.5266,1.8119,...,2.8301,2.2723,2.7572,1.0222,1.1447,1.7660,0.9716,0.8805,0.8246,0.7832
2,ACAA1,6.4613,6.3759,5.6124,5.0930,4.6259,5.9072,5.7616,4.5330,6.0470,...,4.3695,3.8827,4.6399,6.1488,5.5142,4.7587,6.9933,6.9110,6.4041,6.0798
3,ACACA,4.7005,3.7678,4.3385,4.9510,4.0926,3.3307,3.3938,4.5910,4.3060,...,4.3924,4.5224,3.8611,3.4411,4.4101,3.1620,5.6038,6.1530,6.7101,5.6062
4,ACACB,2.0707,0.9493,0.6969,0.4967,0.1257,-0.3566,-0.7588,-0.6193,1.4704,...,0.1776,-0.3566,0.7233,0.7664,0.2029,3.6905,1.6374,1.6187,2.0076,2.2633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,ZFYVE16,3.3321,3.1653,3.1278,2.7594,2.8819,3.1360,2.6278,2.3308,3.1475,...,3.2080,2.0430,2.7336,3.0038,3.1620,2.9875,3.1523,3.4183,3.3321,3.7094
2113,ZFYVE9,2.0395,3.3856,0.7579,0.0990,1.6234,1.2756,1.5758,1.8078,3.1733,...,2.1988,2.7889,2.3479,1.3109,2.4412,2.1541,2.3048,3.1028,3.6042,2.7465
2114,ZMAT3,2.4623,1.5013,1.9786,1.9601,1.5013,0.5470,1.9675,1.4174,3.8471,...,1.7273,3.2870,0.8726,1.9111,0.2762,2.8974,1.2333,0.7999,1.6964,1.2023
2115,ZNF274,3.5742,3.3911,3.4451,3.3675,3.3364,3.3773,3.3278,1.9675,2.6873,...,3.6520,2.5658,2.1047,2.8137,3.4700,2.7292,3.2617,2.9013,3.1844,3.3134


In [92]:
protein_filtered = protein_filtered[['gene_name'] + sorted_patient_sample_list].sort_values(by='gene_name').reset_index(drop=True)
protein_filtered

,gene_name,TCGA-05-4384-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4417-01,TCGA-05-5423-01,TCGA-05-5429-01,TCGA-05-5715-01,TCGA-06-0125-01,...,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,TCGA-ZA-A8F6-01,TCGA-ZG-A8QW-01,TCGA-ZG-A8QX-01,TCGA-ZG-A8QY-01,TCGA-ZG-A8QZ-01
0,ABL1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABL2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACAA1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACACA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACACB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,ZFYVE16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2113,ZFYVE9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2114,ZMAT3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2115,ZNF274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
immune_subtype_filtered = immune_subtype_filtered.sort_values(by='sample').reset_index(drop=True)
immune_subtype_filtered

,sample,Subtype_Immune_Model_Based
0,TCGA-05-4384-01,Inflammatory (Immune C3)
1,TCGA-05-4396-01,Inflammatory (Immune C3)
2,TCGA-05-4405-01,Inflammatory (Immune C3)
3,TCGA-05-4410-01,Inflammatory (Immune C3)
4,TCGA-05-4417-01,Inflammatory (Immune C3)
...,...,...
3587,TCGA-ZA-A8F6-01,Inflammatory (Immune C3)
3588,TCGA-ZG-A8QW-01,Inflammatory (Immune C3)
3589,TCGA-ZG-A8QX-01,Inflammatory (Immune C3)
3590,TCGA-ZG-A8QY-01,Inflammatory (Immune C3)


In [94]:
survival_filtered = survival_filtered.sort_values(by='sample').reset_index(drop=True)
survival_filtered

,sample,_PATIENT,cancer type abbreviation,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,histological_type,initial_pathologic_dx_year,birth_days_to,vital_status,tumor_status,last_contact_days_to,OS,OS.time,DSS,DSS.time,PFI,PFI.time
0,TCGA-05-4384-01,TCGA-05-4384,LUAD,66.0,MALE,NaN,Stage IIIA,Lung Adenocarcinoma,2009.0,-24411.0,Alive,WITH TUMOR,426.0,0.0,426.0,0.0,426.0,1.0,183.0
1,TCGA-05-4396-01,TCGA-05-4396,LUAD,76.0,MALE,NaN,Stage IIIB,Lung Adenocarcinoma,2006.0,-28094.0,Dead,NaN,NaN,1.0,303.0,NaN,303.0,0.0,303.0
2,TCGA-05-4405-01,TCGA-05-4405,LUAD,74.0,FEMALE,NaN,Stage IB,Lung Adenocarcinoma,2006.0,-27241.0,Alive,TUMOR FREE,610.0,0.0,610.0,0.0,610.0,0.0,610.0
3,TCGA-05-4410-01,TCGA-05-4410,LUAD,62.0,MALE,NaN,Stage IB,Lung Adenocarcinoma,2007.0,-22888.0,Alive,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TCGA-05-4417-01,TCGA-05-4417,LUAD,51.0,FEMALE,NaN,Stage IB,Lung Adenocarcinoma,2008.0,-18780.0,Alive,TUMOR FREE,455.0,0.0,455.0,0.0,455.0,0.0,455.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587,TCGA-ZA-A8F6-01,TCGA-ZA-A8F6,STAD,71.0,MALE,WHITE,Stage IB,"Stomach, Intestinal Adenocarcinoma, Not Otherw...",2013.0,-26122.0,Alive,TUMOR FREE,525.0,0.0,525.0,0.0,525.0,0.0,525.0
3588,TCGA-ZG-A8QW-01,TCGA-ZG-A8QW,PRAD,72.0,MALE,NaN,NaN,Prostate Adenocarcinoma Acinar Type,2013.0,-26632.0,Alive,TUMOR FREE,94.0,0.0,94.0,0.0,94.0,0.0,94.0
3589,TCGA-ZG-A8QX-01,TCGA-ZG-A8QX,PRAD,56.0,MALE,NaN,NaN,Prostate Adenocarcinoma Acinar Type,2013.0,-20683.0,Alive,TUMOR FREE,442.0,0.0,442.0,0.0,442.0,0.0,442.0
3590,TCGA-ZG-A8QY-01,TCGA-ZG-A8QY,PRAD,67.0,MALE,NaN,NaN,Prostate Adenocarcinoma Acinar Type,2013.0,-24731.0,Alive,WITH TUMOR,404.0,0.0,404.0,0.0,404.0,0.0,404.0


In [95]:
dense_filtered = dense_filtered.sort_values(by='sample').reset_index(drop=True)
dense_filtered

,sample,sample_type_id,sample_type,_primary_disease
0,TCGA-05-4384-01,1.0,Primary Tumor,lung adenocarcinoma
1,TCGA-05-4396-01,1.0,Primary Tumor,lung adenocarcinoma
2,TCGA-05-4405-01,1.0,Primary Tumor,lung adenocarcinoma
3,TCGA-05-4410-01,1.0,Primary Tumor,lung adenocarcinoma
4,TCGA-05-4417-01,1.0,Primary Tumor,lung adenocarcinoma
...,...,...,...,...
3587,TCGA-ZA-A8F6-01,1.0,Primary Tumor,stomach adenocarcinoma
3588,TCGA-ZG-A8QW-01,1.0,Primary Tumor,prostate adenocarcinoma
3589,TCGA-ZG-A8QX-01,1.0,Primary Tumor,prostate adenocarcinoma
3590,TCGA-ZG-A8QY-01,1.0,Primary Tumor,prostate adenocarcinoma


In [96]:
cellsub_filtered = cellsub_filtered.sort_values(by='sample').reset_index(drop=True)
cellsub_filtered

,sample,Subtype_mRNA,Subtype_Selected
0,TCGA-05-4384-01,NaN,LUAD.1
1,TCGA-05-4396-01,NaN,LUAD.4
2,TCGA-05-4405-01,NaN,LUAD.3
3,TCGA-05-4410-01,NaN,LUAD.3
4,TCGA-05-4417-01,NaN,LUAD.6
...,...,...,...
3587,TCGA-ZA-A8F6-01,NaN,GI.CIN
3588,TCGA-ZG-A8QW-01,1,PRAD.8-other
3589,TCGA-ZG-A8QX-01,1,PRAD.5-SPOP
3590,TCGA-ZG-A8QY-01,3,PRAD.5-SPOP


### 6.2 Create output folder and save processed datasets

In [97]:
import os

# outputfile name
output_folder = 'UCSC-process'
# create folder if not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [98]:
# DataFrame needed to be saved
dataframes = {
    'gene-tran-list.csv': sorted_gene_tran_df,
    'gene-methy-list.csv': sorted_gene_methy_df,
    'gene-protein-list.csv': sorted_gene_protein_df,
    'gene-all-list.csv': sorted_all_gene_df,
    'gene-kegg-edge-list.csv': filtered_up_kegg_df,
    'gene-kegg-path-edge-list.csv': filtered_up_kegg_path_df,
    # 'gene-biogrid-edge-list.csv': filtered_up_biogrid_df,
    # 'gene-string-edge-list.csv': filtered_up_string_df,
    'patient-sample-list.csv': sorted_patient_sample_df,
    'phenotype-lists.csv': phenotype_lists,
    'processed-genotype-methy-Upstream.csv': Upstream_df_filtered,
    'processed-genotype-methy-Distal-Promoter.csv': Distal_Promoter_df_filtered,
    'processed-genotype-methy-Proximal-Promoter.csv': Proximal_Promoter_df_filtered,
    'processed-genotype-methy-Core-Promoter.csv': Core_Promoter_df_filtered,
    'processed-genotype-methy-Downstream.csv': Downstream_df_filtered,
    'processed-genotype-mutation.csv': mutation_filtered,
    'processed-genotype-gene-expression.csv': gene_expression_filtered,
    'processed-genotype-proteomics.csv': protein_filtered,
    'processed-phenotype-immune-subtype-transposed.csv': immune_subtype_filtered,
    'processed-phenotype-survival-transposed.csv': survival_filtered,
    'processed-phenotype-dense-transposed.csv': dense_filtered,
    'processed-phenotype-cellsub-transposed.csv': cellsub_filtered
}

# save to output folder
for file_name, df in dataframes.items():
    df.to_csv(os.path.join(output_folder, file_name), index=False)

c:\Users\hemingzhang\AppData\Local\anaconda3\envs\win_env\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


## 7.Convert the processed data into node dictionary

In [2]:
# load processed data
import pandas as pd
import os

# read the file names under the folder
# Define the path to the output folder where CSV files are stored
output_folder = 'UCSC-process'

# List of file names you saved earlier
file_names = [
    'gene-tran-list', 'gene-methy-list', 'gene-protein-list', 'gene-all-list', 
    'gene-kegg-edge-list', 'gene-kegg-path-edge-list', 
    # 'gene-biogrid-edge-list',
    # 'gene-string-edge-list',
    'patient-sample-list', 'phenotype-lists', 'processed-genotype-methy-Upstream', 
    'processed-genotype-methy-Distal-Promoter', 
    'processed-genotype-methy-Proximal-Promoter', 
    'processed-genotype-methy-Core-Promoter', 'processed-genotype-methy-Downstream', 
    'processed-genotype-mutation', 'processed-genotype-gene-expression', 
    'processed-genotype-proteomics', 'processed-phenotype-immune-subtype-transposed', 
    'processed-phenotype-survival-transposed', 'processed-phenotype-dense-transposed', 
    'processed-phenotype-cellsub-transposed'
]

# Dictionary to hold the dataframes
dataframes = {}

# Read each file and assign to a dataframe
for file_name in file_names:
    full_path = os.path.join(output_folder, file_name + '.csv')
    dataframes[file_name] = pd.read_csv(full_path)

In [3]:
# Assign each dataframe to a variable
sorted_gene_tran_df = dataframes['gene-tran-list']
sorted_gene_methy_df = dataframes['gene-methy-list']
sorted_gene_protein_df = dataframes['gene-protein-list']
sorted_all_gene_df = dataframes['gene-all-list']
filtered_up_kegg_df = dataframes['gene-kegg-edge-list']
filtered_up_kegg_path_df = dataframes['gene-kegg-path-edge-list']
# filtered_up_biogrid_df = dataframes['gene-biogrid-edge-list']
# filtered_up_string_df = dataframes['gene-string-edge-list']
sorted_patient_sample_df = dataframes['patient-sample-list']
phenotype_lists = dataframes['phenotype-lists']
Upstream_df_filtered = dataframes['processed-genotype-methy-Upstream']
Distal_Promoter_df_filtered = dataframes['processed-genotype-methy-Distal-Promoter']
Proximal_Promoter_df_filtered = dataframes['processed-genotype-methy-Proximal-Promoter']
Core_Promoter_df_filtered = dataframes['processed-genotype-methy-Core-Promoter']
Downstream_df_filtered = dataframes['processed-genotype-methy-Downstream']
copynumber_filtered = dataframes['processed-genotype-mutation']
gene_expression_filtered = dataframes['processed-genotype-gene-expression']
protein_filtered = dataframes['processed-genotype-proteomics']
immune_subtype_filtered = dataframes['processed-phenotype-immune-subtype-transposed']
survival_filtered = dataframes['processed-phenotype-survival-transposed']
dense_filtered = dataframes['processed-phenotype-dense-transposed']
cellsub_filtered = dataframes['processed-phenotype-cellsub-transposed']

In [4]:
# outputfile name
graph_output_folder = 'UCSC-graph-data'
# create folder if not exist
if not os.path.exists(graph_output_folder):
    os.makedirs(graph_output_folder)

### 7.1 Make nodes dictionary

In [5]:
sorted_all_gene_dict = sorted_all_gene_df['Gene'].to_dict()
sorted_all_gene_name_dict = {value: key for key, value in sorted_all_gene_dict.items()}
num_gene = sorted_gene_tran_df.shape[0]
num_gene_protein = sorted_gene_protein_df.shape[0]
nodetype_list = ['Gene-TRAN'] * num_gene + ['Gene-METH'] * num_gene + ['Gene-PROT'] * num_gene_protein
map_all_gene_df = pd.DataFrame({'Gene_num': sorted_all_gene_dict.keys(), 'Gene_name': sorted_all_gene_dict.values(), 'NodeType': nodetype_list})
display(map_all_gene_df)
map_all_gene_df.to_csv(os.path.join(graph_output_folder, 'map-all-gene.csv'), index=False)

,Gene_num,Gene_name,NodeType
0,0,ABL1-TRAN,Gene-TRAN
1,1,ABL2-TRAN,Gene-TRAN
2,2,ACAA1-TRAN,Gene-TRAN
3,3,ACACA-TRAN,Gene-TRAN
4,4,ACACB-TRAN,Gene-TRAN
...,...,...,...
6346,6346,ZFYVE16-PROT,Gene-PROT
6347,6347,ZFYVE9-PROT,Gene-PROT
6348,6348,ZMAT3-PROT,Gene-PROT
6349,6349,ZNF274-PROT,Gene-PROT


In [22]:
map_text_all_gene_df = map_all_gene_df.copy()
map_text_all_gene_df['Textual_gene_name'] = map_text_all_gene_df['Gene_name'].str.replace('-METH', ' promoter', regex=False)
map_text_all_gene_df['Textual_gene_name'] = map_text_all_gene_df['Textual_gene_name'].str.replace('-PROT', ' protein', regex=False)
map_text_all_gene_df['Textual_gene_name'] = map_text_all_gene_df['Textual_gene_name'].str.replace('-TRAN', '', regex=False)
display(map_text_all_gene_df)

,Gene_num,Gene_name,NodeType,Textual_gene_name
0,0,ABL1-TRAN,Gene-TRAN,ABL1
1,1,ABL2-TRAN,Gene-TRAN,ABL2
2,2,ACAA1-TRAN,Gene-TRAN,ACAA1
3,3,ACACA-TRAN,Gene-TRAN,ACACA
4,4,ACACB-TRAN,Gene-TRAN,ACACB
...,...,...,...,...
6346,6346,ZFYVE16-PROT,Gene-PROT,ZFYVE16 protein
6347,6347,ZFYVE9-PROT,Gene-PROT,ZFYVE9 protein
6348,6348,ZMAT3-PROT,Gene-PROT,ZMAT3 protein
6349,6349,ZNF274-PROT,Gene-PROT,ZNF274 protein


### 7.2 Create the edges connection between promoter methylations and proteins

In [9]:
# [Gene-METH - Gene]
sorted_gene_methy = sorted_gene_methy_df['Gene'].tolist()
sorted_gene_list = sorted_gene_tran_df['Gene'].tolist()
sorted_gene_protein = sorted_gene_protein_df['Gene'].tolist()
sorted_intersection = [gene_protein.replace('-PROT', '-TRAN') for gene_protein in sorted_gene_protein]
gene_meth_edge_df = pd.DataFrame({'src': sorted_gene_methy, 'dest': sorted_gene_list})
display(gene_meth_edge_df)
# [Gene - Gene-PROT]
gene_protein_edge_df = pd.DataFrame({'src': sorted_intersection, 'dest': sorted_gene_protein})
display(gene_protein_edge_df)

,src,dest
0,ABL1-METH,ABL1-TRAN
1,ABL2-METH,ABL2-TRAN
2,ACAA1-METH,ACAA1-TRAN
3,ACACA-METH,ACACA-TRAN
4,ACACB-METH,ACACB-TRAN
...,...,...
2112,ZFYVE16-METH,ZFYVE16-TRAN
2113,ZFYVE9-METH,ZFYVE9-TRAN
2114,ZMAT3-METH,ZMAT3-TRAN
2115,ZNF274-METH,ZNF274-TRAN


,src,dest
0,ABL1-TRAN,ABL1-PROT
1,ABL2-TRAN,ABL2-PROT
2,ACAA1-TRAN,ACAA1-PROT
3,ACACA-TRAN,ACACA-PROT
4,ACACB-TRAN,ACACB-PROT
...,...,...
2112,ZFYVE16-TRAN,ZFYVE16-PROT
2113,ZFYVE9-TRAN,ZFYVE9-PROT
2114,ZMAT3-TRAN,ZMAT3-PROT
2115,ZNF274-TRAN,ZNF274-PROT


In [104]:
print(sorted_all_gene_name_dict['ABL1-TRAN'])
print(sorted_all_gene_name_dict['ABL1-METH'])
print(sorted_all_gene_name_dict['ABL1-PROT'])

0
2117
4234


In [105]:
# replace gene name with gene number
gene_meth_num_edge_df = gene_meth_edge_df.copy()
gene_meth_num_edge_df['src'] = gene_meth_edge_df['src'].map(sorted_all_gene_name_dict)
gene_meth_num_edge_df['dest'] = gene_meth_edge_df['dest'].map(sorted_all_gene_name_dict)
display(gene_meth_num_edge_df)
gene_protein_num_edge_df = gene_protein_edge_df.copy()
gene_protein_num_edge_df['src'] = gene_protein_edge_df['src'].map(sorted_all_gene_name_dict)
gene_protein_num_edge_df['dest'] = gene_protein_edge_df['dest'].map(sorted_all_gene_name_dict)
display(gene_protein_num_edge_df)

,src,dest
0,2117,0
1,2118,1
2,2119,2
3,2120,3
4,2121,4
...,...,...
2112,4229,2112
2113,4230,2113
2114,4231,2114
2115,4232,2115


,src,dest
0,0,4234
1,1,4235
2,2,4236
3,3,4237
4,4,4238
...,...,...
2112,2112,6346
2113,2113,6347
2114,2114,6348
2115,2115,6349


### 7.3 Concat all of the edges

In [106]:
if selected_database == 'KEGG':
    filtered_up_num_df = filtered_up_kegg_df.copy()
elif selected_database == 'BioGRID':
    filtered_up_num_df = filtered_up_biogrid_df.copy()
elif selected_database == 'STRING':
    filtered_up_num_df = filtered_up_string_df.copy()

# Add 'PROT' to the end of each gene name in the 'src' and 'dest' columns
filtered_up_num_df['src'] = filtered_up_num_df['src'].apply(lambda x: x + '-PROT')
filtered_up_num_df['dest'] = filtered_up_num_df['dest'].apply(lambda x: x + '-PROT')

filtered_up_num_df['src'] = filtered_up_num_df['src'].map(sorted_all_gene_name_dict)
filtered_up_num_df['dest'] = filtered_up_num_df['dest'].map(sorted_all_gene_name_dict)
display(filtered_up_num_df)
all_gene_edge_num_df = pd.concat([filtered_up_num_df, gene_meth_num_edge_df, gene_protein_num_edge_df])
display(all_gene_edge_num_df)

num_gene_edge = filtered_up_num_df.shape[0]
num_gene_meth_edge = gene_meth_num_edge_df.shape[0]
num_gene_protein_edge = gene_protein_num_edge_df.shape[0]
edgetype_list = ['Gene-PROT-Gene-PROT'] * num_gene_edge + ['Gene-TRAN-Gene-METH'] * num_gene_meth_edge + ['Gene-TRAN-Gene-PROT'] * num_gene_protein_edge
all_gene_edge_num_df['EdgeType'] = edgetype_list
all_gene_edge_num_df = all_gene_edge_num_df.sort_values(by=['src', 'dest']).reset_index(drop=True)
display(all_gene_edge_num_df)
all_gene_edge_num_df.to_csv(os.path.join(graph_output_folder, 'all-gene-edge-num.csv'), index=False)

,src,dest
0,4252,5531
1,4252,5532
2,4252,5533
3,4252,6142
4,4253,5531
...,...,...
19689,6350,4928
19690,6350,4929
19691,6350,4930
19692,6350,4931


,src,dest
0,4252,5531
1,4252,5532
2,4252,5533
3,4252,6142
4,4253,5531
...,...,...
2112,2112,6346
2113,2113,6347
2114,2114,6348
2115,2115,6349


,src,dest,EdgeType
0,0,4234,Gene-TRAN-Gene-PROT
1,1,4235,Gene-TRAN-Gene-PROT
2,2,4236,Gene-TRAN-Gene-PROT
3,3,4237,Gene-TRAN-Gene-PROT
4,4,4238,Gene-TRAN-Gene-PROT
...,...,...,...
23923,6350,4928,Gene-PROT-Gene-PROT
23924,6350,4929,Gene-PROT-Gene-PROT
23925,6350,4930,Gene-PROT-Gene-PROT
23926,6350,4931,Gene-PROT-Gene-PROT


In [107]:
# gene edge interactions without map
all_gene_edge_df = all_gene_edge_num_df.copy()
all_gene_edge_df = all_gene_edge_df.replace(sorted_all_gene_dict)

num_gene_edge = filtered_up_num_df.shape[0]
num_gene_meth_edge = gene_meth_edge_df.shape[0]
num_gene_protein_edge = gene_protein_edge_df.shape[0]
# all_gene_edge_df = all_gene_edge_df.sort_values(by=['src', 'dest']).reset_index(drop=True)
all_gene_edge_df.to_csv(os.path.join(graph_output_folder, 'all-gene-edge.csv'), index=False)
display(all_gene_edge_df)

,src,dest,EdgeType
0,ABL1-TRAN,ABL1-PROT,Gene-TRAN-Gene-PROT
1,ABL2-TRAN,ABL2-PROT,Gene-TRAN-Gene-PROT
2,ACAA1-TRAN,ACAA1-PROT,Gene-TRAN-Gene-PROT
3,ACACA-TRAN,ACACA-PROT,Gene-TRAN-Gene-PROT
4,ACACB-TRAN,ACACB-PROT,Gene-TRAN-Gene-PROT
...,...,...,...
23923,ZNRF3-PROT,FZD5-PROT,Gene-PROT-Gene-PROT
23924,ZNRF3-PROT,FZD6-PROT,Gene-PROT-Gene-PROT
23925,ZNRF3-PROT,FZD7-PROT,Gene-PROT-Gene-PROT
23926,ZNRF3-PROT,FZD8-PROT,Gene-PROT-Gene-PROT


## 8.Load data into graph format

### 8.1 Form up the input samples

recommends the use of the endpoints of OS, PFI, DFI, and DSS for each TCGA cancer type

* OS: overall survial
* PFI: progression-free interval
* DSS: disease-specific survival
* DFI: disease-free interval

In [108]:
survival_filtered

,sample,_PATIENT,cancer type abbreviation,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,histological_type,initial_pathologic_dx_year,birth_days_to,vital_status,tumor_status,last_contact_days_to,OS,OS.time,DSS,DSS.time,PFI,PFI.time
0,TCGA-05-4384-01,TCGA-05-4384,LUAD,66.0,MALE,NaN,Stage IIIA,Lung Adenocarcinoma,2009.0,-24411.0,Alive,WITH TUMOR,426.0,0.0,426.0,0.0,426.0,1.0,183.0
1,TCGA-05-4396-01,TCGA-05-4396,LUAD,76.0,MALE,NaN,Stage IIIB,Lung Adenocarcinoma,2006.0,-28094.0,Dead,NaN,NaN,1.0,303.0,NaN,303.0,0.0,303.0
2,TCGA-05-4405-01,TCGA-05-4405,LUAD,74.0,FEMALE,NaN,Stage IB,Lung Adenocarcinoma,2006.0,-27241.0,Alive,TUMOR FREE,610.0,0.0,610.0,0.0,610.0,0.0,610.0
3,TCGA-05-4410-01,TCGA-05-4410,LUAD,62.0,MALE,NaN,Stage IB,Lung Adenocarcinoma,2007.0,-22888.0,Alive,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TCGA-05-4417-01,TCGA-05-4417,LUAD,51.0,FEMALE,NaN,Stage IB,Lung Adenocarcinoma,2008.0,-18780.0,Alive,TUMOR FREE,455.0,0.0,455.0,0.0,455.0,0.0,455.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3587,TCGA-ZA-A8F6-01,TCGA-ZA-A8F6,STAD,71.0,MALE,WHITE,Stage IB,"Stomach, Intestinal Adenocarcinoma, Not Otherw...",2013.0,-26122.0,Alive,TUMOR FREE,525.0,0.0,525.0,0.0,525.0,0.0,525.0
3588,TCGA-ZG-A8QW-01,TCGA-ZG-A8QW,PRAD,72.0,MALE,NaN,NaN,Prostate Adenocarcinoma Acinar Type,2013.0,-26632.0,Alive,TUMOR FREE,94.0,0.0,94.0,0.0,94.0,0.0,94.0
3589,TCGA-ZG-A8QX-01,TCGA-ZG-A8QX,PRAD,56.0,MALE,NaN,NaN,Prostate Adenocarcinoma Acinar Type,2013.0,-20683.0,Alive,TUMOR FREE,442.0,0.0,442.0,0.0,442.0,0.0,442.0
3590,TCGA-ZG-A8QY-01,TCGA-ZG-A8QY,PRAD,67.0,MALE,NaN,NaN,Prostate Adenocarcinoma Acinar Type,2013.0,-24731.0,Alive,WITH TUMOR,404.0,0.0,404.0,0.0,404.0,0.0,404.0


In [109]:
survival_filtered_feature_df = survival_filtered.copy()
survival_filtered_feature_df = survival_filtered_feature_df[['sample', 'cancer type abbreviation', 'OS', 'vital_status']]
display(survival_filtered_feature_df)

nan_counts = survival_filtered_feature_df.isna().sum()  # or df.isnull()
print(nan_counts)

# Convert 'alive' to 0.0 and 'dead' to 1.0
survival_filtered_feature_df['vital_status'] = survival_filtered_feature_df['vital_status'].map({'Alive': 0.0, 'Dead': 1.0})
display(survival_filtered_feature_df)
survival_filtered_feature_df['OS'] == survival_filtered_feature_df['vital_status']


,sample,cancer type abbreviation,OS,vital_status
0,TCGA-05-4384-01,LUAD,0.0,Alive
1,TCGA-05-4396-01,LUAD,1.0,Dead
2,TCGA-05-4405-01,LUAD,0.0,Alive
3,TCGA-05-4410-01,LUAD,0.0,Alive
4,TCGA-05-4417-01,LUAD,0.0,Alive
...,...,...,...,...
3587,TCGA-ZA-A8F6-01,STAD,0.0,Alive
3588,TCGA-ZG-A8QW-01,PRAD,0.0,Alive
3589,TCGA-ZG-A8QX-01,PRAD,0.0,Alive
3590,TCGA-ZG-A8QY-01,PRAD,0.0,Alive


sample                      0
cancer type abbreviation    0
OS                          0
vital_status                0
dtype: int64


,sample,cancer type abbreviation,OS,vital_status
0,TCGA-05-4384-01,LUAD,0.0,0.0
1,TCGA-05-4396-01,LUAD,1.0,1.0
2,TCGA-05-4405-01,LUAD,0.0,0.0
3,TCGA-05-4410-01,LUAD,0.0,0.0
4,TCGA-05-4417-01,LUAD,0.0,0.0
...,...,...,...,...
3587,TCGA-ZA-A8F6-01,STAD,0.0,0.0
3588,TCGA-ZG-A8QW-01,PRAD,0.0,0.0
3589,TCGA-ZG-A8QX-01,PRAD,0.0,0.0
3590,TCGA-ZG-A8QY-01,PRAD,0.0,0.0


0       True
1       True
2       True
3       True
4       True
        ... 
3587    True
3588    True
3589    True
3590    True
3591    True
Length: 3592, dtype: bool

In [110]:
# Check if each row in Column1 and Column2 have the same value
rows_same = (survival_filtered_feature_df['OS'] == survival_filtered_feature_df['vital_status']).all()
print("All rows have the same value in column 'OS' and column 'vital_status' :", rows_same)

All rows have the same value in column 'OS' and column 'vital_status' : True


In [111]:
survival_filtered_feature_df = survival_filtered_feature_df[['sample', 'OS', 'cancer type abbreviation']]
display(survival_filtered_feature_df)
survival_filtered_feature_df.to_csv(os.path.join(graph_output_folder, 'survival-label.csv'), index=False)

,sample,OS,cancer type abbreviation
0,TCGA-05-4384-01,0.0,LUAD
1,TCGA-05-4396-01,1.0,LUAD
2,TCGA-05-4405-01,0.0,LUAD
3,TCGA-05-4410-01,0.0,LUAD
4,TCGA-05-4417-01,0.0,LUAD
...,...,...,...
3587,TCGA-ZA-A8F6-01,0.0,STAD
3588,TCGA-ZG-A8QW-01,0.0,PRAD
3589,TCGA-ZG-A8QX-01,0.0,PRAD
3590,TCGA-ZG-A8QY-01,0.0,PRAD


### 8.2 Randomize the input label

In [113]:
# Randomize the survival label
def input_random(randomized, graph_output_folder):
    if randomized == True:
        random_survival_filtered_feature_df = survival_filtered_feature_df.sample(frac = 1).reset_index(drop=True)
        random_survival_filtered_feature_df.to_csv(os.path.join(graph_output_folder, 'random-survival-label.csv'), index=False)
    else:
        random_survival_filtered_feature_df = pd.read_csv(os.path.join(graph_output_folder, 'random-survival-label.csv'))
    display(random_survival_filtered_feature_df)

input_random(randomized=False, graph_output_folder='UCSC-graph-data')

,sample,OS,cancer type abbreviation
0,TCGA-D7-6525-01,1.0,STAD
1,TCGA-A2-A0YM-01,0.0,BRCA
2,TCGA-BR-8286-01,0.0,STAD
3,TCGA-P5-A735-01,0.0,LGG
4,TCGA-CD-5801-01,1.0,STAD
...,...,...,...
3587,TCGA-BR-A4J4-01,0.0,STAD
3588,TCGA-46-6025-01,0.0,LUSC
3589,TCGA-06-5418-01,1.0,GBM
3590,TCGA-AR-A24S-01,0.0,BRCA


### 8.3 Split the randomized input into 5-fold

In [114]:
# Split deep learning input into training and test
def split_k_fold(k, graph_output_folder):
    random_survival_filtered_feature_df = pd.read_csv(os.path.join(graph_output_folder, 'random-survival-label.csv'))
    num_points = random_survival_filtered_feature_df.shape[0]
    num_div = int(num_points / k)
    num_div_list = [i * num_div for i in range(0, k)]
    num_div_list.append(num_points)
    # Split [random_survival_filtered_feature_df] into [k] folds
    for place_num in range(k):
        low_idx = num_div_list[place_num]
        high_idx = num_div_list[place_num + 1]
        print('\n--------TRAIN-TEST SPLIT WITH TEST FROM ' + str(low_idx) + ' TO ' + str(high_idx) + '--------')
        split_input_df = random_survival_filtered_feature_df[low_idx : high_idx]
        split_input_df.to_csv(os.path.join(graph_output_folder, 'split-random-survival-label-' + str(place_num + 1) + '.csv'), index=False)
        print(split_input_df.shape)

split_k_fold(k=5, graph_output_folder='UCSC-graph-data')


--------TRAIN-TEST SPLIT WITH TEST FROM 0 TO 718--------
(718, 3)

--------TRAIN-TEST SPLIT WITH TEST FROM 718 TO 1436--------
(718, 3)

--------TRAIN-TEST SPLIT WITH TEST FROM 1436 TO 2154--------
(718, 3)

--------TRAIN-TEST SPLIT WITH TEST FROM 2154 TO 2872--------
(718, 3)

--------TRAIN-TEST SPLIT WITH TEST FROM 2872 TO 3592--------
(720, 3)


### 8.4 Reprocess the edge_index file after loading

In [115]:
import os
import numpy as np
import pandas as pd

graph_output_folder = 'UCSC-graph-data'
form_data_path = './' + graph_output_folder + '/form_data'
edge_index = np.load(form_data_path + '/edge_index.npy')
# Convert the 2D array into a DataFrame
edge_index_df = pd.DataFrame(edge_index.T, columns=['src', 'dest'])

gene_edge_num_df = pd.read_csv(os.path.join(graph_output_folder, 'all-gene-edge-num.csv'))
src_gene_list = list(gene_edge_num_df['src'])
dest_gene_list = list(gene_edge_num_df['dest'])
edgetype_list = list(gene_edge_num_df['EdgeType'])
gene_edge_num_reverse_df = pd.DataFrame({'src': dest_gene_list, 'dest': src_gene_list, 'EdgeType': edgetype_list})
gene_edge_num_all_df = pd.concat([gene_edge_num_df, gene_edge_num_reverse_df]).drop_duplicates().sort_values(by=['src', 'dest']).reset_index(drop=True)

display(edge_index_df)
display(gene_edge_num_all_df)
merged_gene_edge_num_all_df = pd.merge(gene_edge_num_all_df, edge_index_df, on=['src', 'dest'], how='inner')
display(merged_gene_edge_num_all_df)
merged_gene_edge_num_all_df.to_csv(os.path.join(graph_output_folder, 'merged-gene-edge-num-all.csv'), index=False)

merged_gene_edge_name_all_df = merged_gene_edge_num_all_df.replace(sorted_all_gene_dict)
display(merged_gene_edge_name_all_df)
merged_gene_edge_name_all_df.to_csv(os.path.join(graph_output_folder, 'merged-gene-edge-name-all.csv'), index=False)

,src,dest
0,0,2117
1,0,4234
2,1,2118
3,1,4235
4,2,2119
...,...,...
47173,6350,4932
47174,6350,5994
47175,6350,5995
47176,6350,5996


,src,dest,EdgeType
0,0,2117,Gene-TRAN-Gene-METH
1,0,4234,Gene-TRAN-Gene-PROT
2,1,2118,Gene-TRAN-Gene-METH
3,1,4235,Gene-TRAN-Gene-PROT
4,2,2119,Gene-TRAN-Gene-METH
...,...,...,...
47173,6350,4932,Gene-PROT-Gene-PROT
47174,6350,5994,Gene-PROT-Gene-PROT
47175,6350,5995,Gene-PROT-Gene-PROT
47176,6350,5996,Gene-PROT-Gene-PROT


,src,dest,EdgeType
0,0,2117,Gene-TRAN-Gene-METH
1,0,4234,Gene-TRAN-Gene-PROT
2,1,2118,Gene-TRAN-Gene-METH
3,1,4235,Gene-TRAN-Gene-PROT
4,2,2119,Gene-TRAN-Gene-METH
...,...,...,...
47173,6350,4932,Gene-PROT-Gene-PROT
47174,6350,5994,Gene-PROT-Gene-PROT
47175,6350,5995,Gene-PROT-Gene-PROT
47176,6350,5996,Gene-PROT-Gene-PROT


,src,dest,EdgeType
0,ABL1-TRAN,ABL1-METH,Gene-TRAN-Gene-METH
1,ABL1-TRAN,ABL1-PROT,Gene-TRAN-Gene-PROT
2,ABL2-TRAN,ABL2-METH,Gene-TRAN-Gene-METH
3,ABL2-TRAN,ABL2-PROT,Gene-TRAN-Gene-PROT
4,ACAA1-TRAN,ACAA1-METH,Gene-TRAN-Gene-METH
...,...,...,...
47173,ZNRF3-PROT,FZD9-PROT,Gene-PROT-Gene-PROT
47174,ZNRF3-PROT,RSPO1-PROT,Gene-PROT-Gene-PROT
47175,ZNRF3-PROT,RSPO2-PROT,Gene-PROT-Gene-PROT
47176,ZNRF3-PROT,RSPO3-PROT,Gene-PROT-Gene-PROT
